# spin transformations on discrete surfaces
Tim Tyree<br>
12.30.2022

This notebook works along with the paper of the same name by Crane, Pinkall, and Schröder (2011).

In [ ]:
from lib.my_initialization import *

In [ ]:
plt.plot([4]*3)
plt.close()
#reset matplotlib
import matplotlib as mpl
sns.reset_orig()
mpl.rc_file_defaults()
#set randomization seed
np.random.seed(42)
#consider darkmode
darkmode=False
if darkmode:
    # For darkmode plots
    from jupyterthemes import jtplot
    jtplot.style(theme='monokai', context='notebook', ticks=True, grid=False)
    
%load_ext autoreload
%autoreload 2

In [ ]:
import scipy.io as sio
from IPython.display import IFrame

In [ ]:
import scipy
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import lsqr

## scratchwerk

In [ ]:
#TODO(later): try getting pyrenderer working on macintosh with the supposed OpenGL support 

In [ ]:
# fullName = '/System/Library/Frameworks/OpenGL.framework/OpenGL'
# #TODO: try to use this? does ^this help?

In [ ]:
# import os
# # switch to "osmesa" or "egl" before loading pyrender
# # os.environ["PYOPENGL_PLATFORM"] = "egl"
# os.environ["PYOPENGL_PLATFORM"] = "osmesa"

# import numpy as np
# import pyrender
# import trimesh
# import matplotlib.pyplot as plt

In [ ]:

# # generate mesh
# sphere = trimesh.creation.icosphere(subdivisions=4, radius=0.8)
# sphere.vertices+=1e-2*np.random.randn(*sphere.vertices.shape)
# mesh = pyrender.Mesh.from_trimesh(sphere, smooth=False)

# # compose scene
# scene = pyrender.Scene(ambient_light=[.1, .1, .3], bg_color=[0, 0, 0])
# camera = pyrender.PerspectiveCamera( yfov=np.pi / 3.0)
# light = pyrender.DirectionalLight(color=[1,1,1], intensity=2e3)

# scene.add(mesh, pose=  np.eye(4))
# scene.add(light, pose=  np.eye(4))

# c = 2**-0.5
# scene.add(camera, pose=[[ 1,  0,  0,  0],
#                         [ 0,  c, -c, -2],
#                         [ 0,  c,  c,  2],
#                         [ 0,  0,  0,  1]])

# # render scene
# r = pyrender.OffscreenRenderer(512, 512)
# color, _ = r.render(scene)

# plt.figure(figsize=(8,8)), plt.imshow(color);

# define module

In [ ]:
from PIL import Image
import trimesh
def save_scene_to_png_trimesh(save_fn_img, scene, 
                              resolution = (1280,720), 
                              visible=False, 
                              format_str='PNG',
                              start_loop=False,
                              **kwargs):
    """saves trimesh.Scene instance to .png.
    kwargs are passed to scene.save_image.
    start_loop=True will launch an ipython kernel, which might not be desired.
    
    Example Usage:
camera_angle=(-90-45,0,45)
scene = trimesh.scene.Scene(mesh_current)
scene.set_camera(angles=camera_angle)
save_scene_to_png_trimesh(save_fn_img, scene, 
                          resolution = (1280,720),visible=False, format='PNG',
                          start_loop=False,background='dark')#,**kwargs)
print(f"open {os.path.abspath(save_fn_img)}")
    """
    #save scene as .png
    rendered = Image.open(
        trimesh.util.wrap_as_stream(
            scene.save_image(
                resolution=resolution, visible=visible,
                start_loop=start_loop,
                **kwargs)))
    rendered.save(save_fn_img, format=format_str)
    del rendered
    #TODO(later): figure out how to make scene.save_image() stop launching an ipykernel
    # 	plotter.deep_clean()
    # 	del plotter
    # 	pv.close_all()
    return True

def get_colors_trimesh(inp, colormap=None, vmin=None, vmax=None):
    """inp is a 1D numpy array that is mapped to a plt.cm.jet if colormap is None.
    
    Example Usage:
colors = get_colors_trimesh(rho, plt.cm.jet)
mesh = trimesh.Trimesh(vertices=vert,faces=tria,face_colors=colors)
    """
    
    if colormap is None:
        colormap = plt.cm.jet
    # norm = plt.Normalize()
    # colors = plt.cm.jet(norm(rho))
    # colors = plt.cm.jet(np.linspace(0,1,len(dz)))
    norm = plt.Normalize(vmin, vmax)
    colors = colormap(norm(inp))
    return (255*colors/np.max(colors)).astype('uint8')#[:,0,:]

def save_trimesh_to_png(scene,save_fn_img,
                        format_str='PNG',
                        resolution = (1280,720), 
                        visible=True,
                        start_loop=False,
                        use_dispatch_events_extra=True,
#                         use_wireframe=True,
                        **kwargs):
    """"
    save_trimesh_to_png saves scene to save_fn_img in format_str format.
    scene is a trimesh.scene.scene.Scene instance.
    additional kwargs are passed to trimesh.viewer.windowed.SceneViewer.
    
    Example Usage:
save_trimesh_to_png(scene,save_fn_img,format_str='PNG')
print(f"open {os.path.abspath(save_fn_img)}")
    """
    #initialize the window
    window = trimesh.viewer.windowed.SceneViewer(scene, resolution = resolution, format=format_str,
                                                 visible=visible,start_loop=start_loop,**kwargs)
#     if use_wireframe:
#         window.toggle_wireframe()
#didn't help...
#     @window.event
#     def on_draw():
#         window.scene.render()
#         #window.render()
        
    if use_dispatch_events_extra:
        #is this fixed?
        window.dispatch_events() #<<< first of three calls will liklely send the warning messages

        # need to run loop twice to display anything
        for save in [False, False, True]:
            window.dispatch_events() #needed #src of warning messages
            window.dispatch_event('on_draw') #needed
            if save:
                # save the color buffer data to memory
                file_obj = trimesh.viewer.windowed.util.BytesIO()
                window.save_image(file_obj)
                file_obj.seek(0)
                png = file_obj.read()
        window.close()
        rendered = Image.open(trimesh.util.wrap_as_stream(png))
        #save rendered scene as .png
        rendered.save(save_fn_img, format=format_str)
        del rendered, file_obj, png
        return True

In [ ]:
# rendered = Image.open(
#     trimesh.util.wrap_as_stream(
#         scene.save_image()#resolution=resolution, visible=visible,**kwargs)))

In [ ]:
@njit
def jiH(pnt):
    """jiH castes a cartesion point, pnt, to a real quaternionic representation."""
    a=pnt[0]; b=pnt[1]; c=pnt[2]; d=pnt[3];
    h = np.array([
        [a, -b, -c, -d],
        [b,  a, -d,  c],
        [c,  d,  a, -b],
        [d, -c,  b,  a]])
    return h


In [ ]:
def map_lam_real(lam,V,T,printing=True,**kwargs):
    """map_lam_real maps the (eigen)vector, lam, from the real representation of vector-quaternions to 
    vetex representation of map, L,ome.
    
    Example Usage:
L,ome=map_lam_real(lam,V,T,printing=True)#,**kwargs)
    """
    #input: V,T,lam
    #output: L,ome
    nT=T.shape[0]
    nV=V.shape[0]
    #transcribe eigensolution to output mesh.
    ome=np.zeros(4*nV)
    # L  =scipy.sparse.csc_matrix(4*nV,4*nV) #<<Q: is this sparse faster?
    L  =np.zeros((4*nV,4*nV))
    # L  =sparse(4*nV,4*nV);
    num_steps=nT-1
    update_printbar_every=int(np.around(nT/20))
    step=0
    # for c1 in range(1,nT): # for c1=1:nT
    # for c1 in range(nT+1): # for c1=1:nT #<<< Index Error
    # for c1 in range(nT-1): # for c1=1:nT
    # #Q: is ^this the problem?
    # #A: it appears not...
    for c1 in range(nT): # for c1=1:nT
        for c2 in range(3): #for c2=1:3
            k0 = T[c1,(c2-1)%3]
            k1 = T[c1,(c2+0)%3]
            k2 = T[c1,(c2+1)%3]
            u1=V[k1]-V[k0]
            u2=V[k2]-V[k0]
            cta = np.dot (u1,u2) / np.linalg.norm (np.cross (u1,u2) )
            h=jiH(np.array([0.5*cta, 0, 0, 0]));
            # write to global vertex representation
            ooow = np.hstack([np.concatenate([h, -h]),
                              np.concatenate([-h, h])]) # [[h -h],-[h h]]
            ini=np.hstack([k1*4+plc,  k2*4+plc])
            # #ini=np.hstack([k2*4+plc,  k1*4+plc]) #global swap?
            L[np.ix_(ini,ini)] += ooow  # L(ini,ini) = L(ini,ini) + [[h -h],-[h h]]
            # ooow_= np.concatenate([ooow[1],ooow[2],ooow[0]],axis=2)
            # oooow = np.vstack([ooow_[1],ooow_[2],ooow_[0]])
            #L[np.ix_(ini,ini)] += oooow
            if k1>k2: #swap s.t. k3 is tmp
                k3=k1; k1=k2; k2=k3;
            lm1=jiH(lam[k1*4+plc])
            lm2=jiH(lam[k2*4+plc])
            edv=jiH(np.concatenate([np.array([0]),V[k2]-V[k1]]))
    #         ti1 = (lm1*edv)@lm1/3.   + (lm2*edv)@lm2/3.   + (lm1*edv)@lm2/6. + (lm2*edv)@lm1/6.
    #         ti1 = lm1*edv@lm1/3.   + lm2*edv@lm2/3.   + lm1*edv@lm2/6. + lm2*edv@lm1/6.
    #         ti1 = lm1*(edv@lm1)/3. + lm2*(edv@lm2)/3. + lm1*(edv@lm2)/6. + lm2*(edv@lm1)/6.
    #         ti1 = lm1@edv@lm1/3. + lm2@edv@lm2/3. + lm1@edv@lm2/6. + lm2@edv@lm1/6.
            #ti1 = (lm1@edv)@lm1/3.   + (lm2@edv)@lm2/3.   + (lm1@edv)@lm2/6. + (lm2@edv)@lm1/6.# <<< clearly wrong
            ti1 = (lm1.T@edv)@lm1/3.   + (lm2.T@edv)@lm2/3.   + (lm1.T@edv)@lm2/6. + (lm2.T@edv)@lm1/6.
            ti1 *= -1.
            #Q: which of ^these is right?
            #A: left to right, * and then @
            # til=lm1'*edv*lm1/3 + lm1'*edv*lm2/6 + lm2'*edv*lm1/6 + lm2'*edv*lm2/3;
    #         ome[k1*4+plc] = ome[k1*4+plc]-0.5*cta*ti1[0] #<<<wrong sign for rotations
            ome[k1*4+plc] = ome[k1*4+plc]-0.5*cta*ti1[:,0]  #<<< this one agrees with matlab? it looks like it gives the wrong sign to rotation
            #Q: which of ^these is right?
            # ome(k1*4+plc,1)=ome(k1*4+plc,1)-cta*til(:,1)/2;
    #         ome[k2*4+plc] = ome[k2*4+plc]+0.5*cta*ti1[0]#<<<wrong sign for rotations
            ome[k2*4+plc] = ome[k2*4+plc]+0.5*cta*ti1[:,0]  #<<< this one agrees with matlab? it looks like it gives the wrong sign to rotation
            #Q: which of ^these is right?
            # ome(k2*4+plc,1)=ome(k2*4+plc,1)+cta*til(:,1)/2;
            #Q: am I certain that i use ome[k2*4+plc] and not ome[k2*4+plc,0]??
        #print progress bar
        if printing:
            step+=1
            if step%update_printbar_every==0:
                printProgressBar(step+1, num_steps, prefix = 'Progress:', suffix = 'Complete', length = 50)
    return L,ome

In [ ]:
def map_real_lam_to_vertices(lam,V,T,printing=True,use_return_Lp=True,**kwargs):
    """map_real_lam_to_vertices does essentially a linear algebra solve to 
    - compute the linear map of the solution vector, lam,
    - on triangular mesh with 
        - vertices, V (N,3 numpy.array), and
        - faces, T (N,3, numpy.array)   
    is this really 2 seconds per linear algebra solve? yes. 
    UNLESS if you're out of virtual memory stored in swap ram.

    minimizes the least square of $ | Lp . x - ome | ^ 2 $
    
    kwargs are passed to scipy.sparse.linalg.lsqr

    Example Usage:
x,Lp,ome = map_real_lam_to_vertices(lam,V,T)#,printing=True,use_return_Lp=True,**kwargs)
    """
    L,ome=map_lam_real(lam,V,T,printing=printing)#,**kwargs)
    # print(f"{np.around(L[:10,:10],2)=}")
    # print(f"{ome[:6]=}")
    Lp=np.roll(np.roll(L,3,axis=0),3,axis=1) #bc of plc indexing...
    omep=np.roll(ome,3) #bc of plc indexing...
    #center input
    ome = omep.reshape((nV,4))
    ome = ome - np.broadcast_to(np.mean(ome,axis=0), shape=(nV,4), subok=False) #note subok=True would be needed for numpy-quaternion entries in ome.
    ome = ome.flatten()
    A = csc_matrix(Lp, dtype=float) #<<< looks fastest for demo rho
    # A = scipy.sparse.bsr_matrix(Lp, dtype=float) #<<< alternative sparse repr
    # A = scipy.sparse.csr_matrix(Lp, dtype=float) #<<< alternative sparse repr
    #solve the linear system in the least square
    b = np.array(ome, dtype=float)
    if printing:
        print(f"\nPerforming linear algebra solve...")
    x, istop, itn, normr = scipy.sparse.linalg.lsqr(A, b, **kwargs)[:4]
    if use_return_Lp:
        if printing:
            print(f"linear solve complete!\nthe number of iterations used for linear algebra solve: {istop=}")
            print(f"returning Lp,ome: {Lp.shape=}, {ome.shape=}")
            print(f"{np.min(L)=:7f},{np.max(L)=:7f}")
            print(f"{np.min(ome)=:7f},{np.max(ome)=:7f}")
        return x,Lp,ome
    else:
        return x,None,None
    

In [ ]:
def comp_groundstate_eigenvector_sparse(E):
    """comp_groundstate_eigenvector_sparse 
    appears faster than banded solvers...
    
    Example Usage:
lam,cnv = comp_groundstate_eigenvector_sparse(E)
    
    """
    # Eab = decomp_banded_matrix(E)
    E_sparse=scipy.sparse.csc_matrix(E,dtype=float)
    lam=np.zeros(4*nV)
    lam[::4]=1.
    b=lam
    # num_steps=11
    # for step in range(num_steps):
    #     cnv=lam
    #Q: what's the fastest way to compute x vs. x2 or x3
    x2 =scipy.sparse.linalg.lsqr(E_sparse,b)[0] #lam=lam/E
    # x3 =solveh_banded(Eab,b)#,lower=False)#,**kwargs) #lam =mldivide(lam,ab)
    #Q: what's the fastest way to compute x vs. x2 or x3
    # A: x2
    lam=x2.copy()
    # lam=x3.copy()
    cnv=b.copy()
    return lam,cnv

In [ ]:
def comp_groundstate_eigenvector_inverse(E,num_steps=11,printing=True):
    """computes lam = lam . Einv num_steps times.
    comp_groundstate_eigenvector_inverse has a relatively slow run time...
    
    Example Usage:
lam,cnv,Einv = comp_groundstate_eigenvector_inverse(E)
res=(lam@E)/lam; 
print(f'mean: {np.mean(res):e}, var:  {np.var(res):e}, delta: {np.linalg.norm(cnv-lam):e}')
    """
    #sparse soln:
    # mean: -1.825385e-04, var:  1.688050e-04, delta: 2.156203e+05
    # Q: how does lam look?
    # A: great! 
    #banded soln: ?? second run time.... does it crash the kernel?
    # mean: ??
    # Q: how does lam look?
    # A: ??
    #omverse soln:
    #3 minute runtime for 11 epochs
    #isn't this supposed to be faster
    #input: E,nV=E.shape[0]/4
    #output: lam
    lam=np.zeros(4*nV)
    lam[::4]=1.
    # lam[1::4]=1.
    # lam[-1::4]=1.
    # lam[3::4]=1.
    # lam/=np.linalg.norm(lam)
    # lam+=1.
    #Q: is ^this right?
    #Q is this better? 
    # ab = decomp_banded_matrix(E)
    # ab = decomp_banded_matrix(E)
    # E_sparse=scipy.sparse.csc_matrix(E,dtype=float)
    Einv=np.linalg.inv(E)
    for step in range(num_steps):
        cnv=lam

        lam = lam@Einv
        #lam =scipy.sparse.linalg.lsqr(E_sparse,lam)[0] #lam=lam/E
        #lam =solveh_banded(ab,lam)#,lower=False)#,**kwargs) #lam =mldivide(lam,ab)
        lam/=np.linalg.norm(lam);
        #print progress bar
        if printing:
            step+=1
            if step%update_printbar_every==0:
                printProgressBar(step+1, num_steps, prefix = 'Progress:', suffix = 'Complete', length = 50)
    return lam,cnv,Einv

In [ ]:
#Q: is this the general identity map? 
#A: yes! yes, it is.
def comp_trimesh_curvature(mesh,radius=1./(2.*np.pi)):
    """
    Example Usage:
rho = comp_trimesh_curvature(mesh,radius=1./(2.*np.pi))
    """
    tria=mesh.faces
    mean_curvature_values = trimesh.curvature.\
                            discrete_mean_curvature_measure(mesh, 
                                                            points=mesh.vertices, 
                                                            radius=radius)
    # rho_vertex_values = mean_curvature_values/radius**2
    rho_vertex_values = mean_curvature_values/radius**2/2
    rho_vertex_values[tria].shape
    #is this the identity map?
    rho = np.mean(rho_vertex_values[tria],axis=1)
    return rho

In [ ]:
##############################################
# note directly used in final implentation
##############################################

In [ ]:
from scipy.linalg import solveh_banded

def decomp_banded_matrix(A):
    """A is an NxN matrix as a 2D numpy array. entries may be objects.
    decomp_banded_matrix returns a 2D numpy array instance with D=the number of nontrivial bands.
    
    Example Usage:
ab = decomp_banded_matrix(A)
ab.shape
    """
    D=np.max(np.count_nonzero(A,axis=1))
    ab = np.zeros((D,A.shape[0]))
    for i in np.arange(1,D):
        ab[i,:] = np.concatenate((np.diag(A,k=i),np.zeros(i,)),axis=None)
    ab[0,:] = np.diag(A,k=0)
    return ab

def mldivide_banded(ab,x,lower=True,**kwargs):
    """
    Example Usage:
#input: E,nV=E.shape[0]/4
lam=np.zeros(4*nV)
lam[::4]=1.
# lam[1::4]=1.
# lam[-1::4]=1.
# lam[3::4]=1.
lam/=np.linalg.norm(lam)
# lam+=1.
#Q: is ^this right?
#Q is this better? 
ab = decomp_banded_matrix(E)
for c1 in range(11):
    cnv=lam
    #lam=lam/E as ab
    lam =solveh_banded(ab,lam,lower=lower,**kwargs) #lam =mldivide(lam,ab)
    lam/=np.linalg.norm(lam);
res=(lam@E)/lam; 
print(f'mean: {np.mean(res):e}, var:  {np.var(res):e}, delta: {np.linalg.norm(cnv-lam):e}')



    """
    return solveh_banded(ab,x,lower=lower,**kwargs)

def invh_banded(A, x):
    N = np.shape(A)[0]
    D = np.count_nonzero(A[0,:])
    ab = np.zeros((D,N))
    for i in np.arange(1,D):
        ab[i,:] = np.concatenate((np.diag(A,k=i),np.zeros(i,)),axis=None)
    ab[0,:] = np.diag(A,k=0)
    y = solveh_banded(ab,x,lower=True)
    return y

# y=invh_banded(E, np.ones())

# def invh_banded_simple(A, x):
#     N = np.shape(A)[0]
#     D = np.count_nonzero(A[0,:])
#     ab = np.zeros((D,N))
#     for i in np.arange(1,D):
#         ab[i,:] = np.concatenate((np.diag(A,k=i),np.zeros(i,)),axis=None)
#     ab[0,:] = np.diag(A,k=0)
#     y = solveh_banded(ab,x,lower=True)
#     return y

In [ ]:
def AddLegend(ax=None,xy = (1.04,1.04),fontsize=15,loc='upper left',frameon=False,**kwargs):
    """AddLegend adds a legend to a matplotlib plot.

    Example Usage:
ax = AddLegend()#ax=None,xy = (1.04,1.04),fontsize=15,loc='upper left',frameon=False,**kwargs)

    Example Usage:
#format_plot
format_plot(xlabel='n',ylabel='Probability')
ax = AddLegend(ax,fontsize=fontsize-3,xy = (0.96,0.96), loc='upper right',frameon=False)#,**kwargs)
    """
    if ax is None:
        ax = plt.gca()
    return ax.legend(fontsize=fontsize,loc=loc, bbox_to_anchor = xy, frameon=frameon, **kwargs)

def format_plot(ax=None,xlabel=None,ylabel=None,fontsize=20,use_loglog=False,xlim=None,ylim=None,use_bigticks=True,**kwargs):
    '''format plot formats the matplotlib axis instance, ax,
    performing routine formatting to the plot,
    labeling the x axis by the string, xlabel and
    labeling the y axis by the string, ylabel
    '''
    if not ax:
        ax=plt.gca()
    if use_loglog:
        ax.set_xscale('log')
        ax.set_yscale('log')
    if xlabel:
        ax.set_xlabel(xlabel,fontsize=fontsize,**kwargs)
    if ylabel:
        ax.set_ylabel(ylabel,fontsize=fontsize,**kwargs)
    if use_bigticks:
        ax.tick_params(axis='both', which='major', labelsize=fontsize,**kwargs)
        ax.tick_params(axis='both', which='minor', labelsize=0,**kwargs)
    if xlim:
        ax.set_xlim(xlim)
    if ylim:
        ax.set_xlim(ylim)
    return True

# deriving the half mean curvature density of the sphere
then,
- TODO: use trimesh to estimate the half mean curvature density of a triangular mesh in the shape of a sphere.
- HINT: use the known spherical case as numerical reference point.


$$\rho(x) =? 2*H_f(x;V,T) =? 2/r =? 2$$

$1+1$

In [ ]:
# mesh.crc
# mesh.edges_unique_inverse
mesh

In [ ]:
# !pip install rtree

In [ ]:
step=0
printing=True
update_printbar_every=2
radius_set_values=np.arange(1e-3,1,5e-3)
num_steps = radius_set_values.shape[0]
mean_curvature_values_lst=[]
for radius in radius_set_values:
    mean_curvature_values = trimesh.curvature.\
                        discrete_mean_curvature_measure(mesh, 
                                                        points=mesh.vertices, 
                                                        radius=radius)
    #record
    mean_curvature_values_lst.append(mean_curvature_values)
    #print progress bar
    if printing:
        step+=1
        if step%update_printbar_every==0:
            printProgressBar(step+1, num_steps, prefix = 'Progress:', suffix = 'Complete', length = 50)
mean_curvature_values.shape,len(mean_curvature_values_lst)

In [ ]:
# H_lst = [np.mean(X) for X in mean_curvature_values_lst]
H_values = np.array([np.mean(X) for X in mean_curvature_values_lst])

In [ ]:
plt.plot(2*H_values,2*np.pi*radius_set_values**2)
format_plot(xlabel=r"$rho=2H_{mean}$",ylabel=r'$2\pi R^2_{sphere}$')
plt.xlim([0,6])
plt.ylim([0,6])
plt.show()

In [ ]:
mesh.vertices -= mesh.center_mass
R = np.linalg.norm(np.mean(mesh.vertices**2,axis=0))
rhobar = np.pi*R**2/2
print(f"{rhobar=}")
#likely more stable and generalizes
rho = 0.5*H_values # (2.*np.pi*R**2)
print(f"{np.mean(rho)=}")

In [ ]:
#is this the general identity map? 
radius=1./(2.*np.pi)
mean_curvature_values = trimesh.curvature.\
                        discrete_mean_curvature_measure(mesh, 
                                                        points=mesh.vertices, 
                                                        radius=radius)
rho = mean_curvature_values/radius**2

In [ ]:
# mesh.mass_properties

In [ ]:
# np.shape(E)

# load example triangle mesh endowed with scalar curvture field

In [ ]:
root_dir = '/'.join(nb_dir.split('/')[:3])
# mesh_folder = f"{root_dir}/Documents/MATLAB/spin/res"
mesh_folder = f"{root_dir}/Documents/meshes/res"
os.chdir(mesh_folder)
for fn in sorted(os.listdir()):
    pass
fn

In [ ]:
fn='sphere.mat'

In [ ]:
#load example mesh with custom curvature field
mat = sio.loadmat(fn)
# tria,vert,rho = mat['model'][0,0]
vert,tria,rho = mat['model'][0,0]
rho = rho.T.flatten()
dtype='float64'
# dtype='float128' #does not support np.linalg.inv...
#use more expensive data types
#(~5 min run time with '<f8')
vert=vert.astype(dtype)
rho=rho.astype(dtype)


rho*=4.
# rho0=rho.copy()

np.min(tria),np.max(tria),tria.shape,rho.shape

In [ ]:
# rho*=0.
# # rho+=0.5

In [ ]:
# rho=0.5*H_values
# rho.shape

In [ ]:

# vert = vert.T.copy()-1 #correct for matlab's 1 indexing
# tria = tria.T.copy()
tria = tria.T.copy()-1 #correct for matlab's 1 indexing
vert = vert.T.copy()
use_whiten_mesh=False
if use_whiten_mesh:
    #optionally whiten the mesh
    vert -= np.mean(vert,axis=0)
    vert /= np.std(vert,axis=0)#.show()
# plt.scatter(x=tria[:,0],y=tria[:,1])
plt.scatter(x=vert[:,0],y=vert[:,2])#,c=rho,cmap='jet')
plt.show()
# vert.shape,tria.shape,rho.shape,np.max(vert)
tria.shape,rho.shape,vert.shape

In [ ]:
colors = get_colors_trimesh(rho, plt.cm.jet)


# # if colormap is None:
# colormap = plt.cm.jet
# norm = plt.Normalize(vmin=None, vmax=None)
# colors = colormap(norm(rho))
# # (255*colors/np.max(colors)).astype('uint8')[:,0,:]
# colors = (255*colors/np.max(colors)).astype('uint8')#[:,0:]


In [ ]:
# initialize mesh colored by rho
mesh = trimesh.Trimesh(vertices=vert,faces=tria,
                       face_colors=colors,
                       face_attributes={'rho':rho})

In [ ]:
# the convex hull is another Trimesh object that is available as a property
# lets compare the volume of our mesh with the volume of its convex hull
print(f"{mesh.volume / mesh.convex_hull.volume=:.7f}")
# # since the mesh is watertight, it means there is a
# # volumetric center of mass which we can set as the origin for our mesh
print(f"{mesh.center_mass=}")
# mesh.vertices -= mesh.center_mass
print(f"{mesh.center_mass=}")
# what's the moment of inertia for the mesh?
print(f"{mesh.moment_inertia=}")

# # if there are multiple bodies in the mesh we can split the mesh by
# # connected components of face adjacency
# # since this example mesh is a single watertight body we get a list of one mesh
# mesh.split()

# axis aligned bounding box is available
print(f"{mesh.bounding_box.extents=}")

In [ ]:
rho0 = comp_trimesh_curvature(mesh,radius=1./(2.*np.pi))


In [ ]:
# trivial setting:
rho = rho0

In [ ]:
# #Q: does dividing rho by the identity rho0 change the shape of the result?
# #A: no. also, it has the same percent contraction as the original rho
# rho /= rho0
# rho.shape

In [ ]:
# # #color vertices manually
# # for i,c in enumerate(vertex_colors):
# #     mesh.visual.vertex_colors[i]=c
# # mesh.visual.vertex_colors
# # #randomize face colors
# for face in mesh.faces:
#     mesh.visual.vertex_colors[face] = colors[face]
# for facet in mesh.facets:
#     mesh.visual.vertex_colors[facet] = trimesh.visual.random_color()
# mesh.show()

In [ ]:
# #plot an image on the mesh
# m = mesh
# uv = np.random.rand(m.vertices.shape[0], 2)
# # im = np.random.rand(m.vertices.shape[0], m.vertices.shape[0])
# im = np.random.rand(200,200)
# material = trimesh.visual.texture.SimpleMaterial(image=im)
# color_visuals = trimesh.visual.TextureVisuals(uv=uv, image=im, material=material)
# mesh=trimesh.Trimesh(vertices=m.vertices, faces=m.faces, visual=color_visuals, validate=True, process=False)
# mesh.show()

In [ ]:
# # transform method can be passed a (4, 4) matrix and will cleanly apply the transform
# mesh.apply_transform(trimesh.transformations.random_rotation_matrix())

# # a minimum volume oriented bounding box also available
# # primitives are subclasses of Trimesh objects which automatically generate
# # faces and vertices from data stored in the 'primitive' attribute
# print(f"{mesh.bounding_box_oriented.primitive.extents=}")
# print(f"{mesh.bounding_box_oriented.primitive.transform=}")

# # bounding spheres and bounding cylinders of meshes are also
# # available, and will be the minimum volume version of each
# # except in certain degenerate cases, where they will be no worse
# # than a least squares fit version of the primitive.
# print(f"""{mesh.bounding_box_oriented.volume=},
#       {mesh.bounding_cylinder.volume=},
#       {mesh.bounding_sphere.volume=}""")

# # show the mesh appended with its oriented bounding box
# # the bounding box is a trimesh.primitives.Box object, which subclasses
# # Trimesh and lazily evaluates to fill in vertices and faces when requested
# # (press w in viewer to see triangles)
# # (mesh + mesh.bounding_box_oriented).show()
# mesh.show()


In [ ]:
#view the mesh
# mesh.show()

#view the mesh
# scene = trimesh.scene.Scene(mesh)
# trimesh.viewer.notebook.scene_to_notebook(scene=scene, height=200)

In [ ]:
#print mesh bluf
print(f"{mesh.volume=}")
print(f"{mesh.center_mass=}")
print(f"{mesh.is_watertight=} (is the current mesh watertight?)")
print(f"{mesh.euler_number=} (what's the euler number for the mesh?)")

In [ ]:
volume_initial = mesh.volume#=4.182994798204928
print(f"{volume_initial=} voxels")

# scratchwerk?

## DONT: generate smaller mesh for dev of real eigensolving
- note: this mesh is singular...

In [ ]:
# # vert = trimesh.sample.sample_surface_sphere(100)

# radius=1.0
# mesh = trimesh.creation.uv_sphere(
#     radius=radius,
#     count=[5, 5],)
# #     count=[10, 10],)
# #     count=[32, 32],)
# mesh.center_mass

# cscale=5
# #caste discrete unit sphere to numpy
# vert = np.array(mesh.vertices)
# tria = np.array(mesh.faces)
# rho  = tria[:,0]*0.+1./(2.*radius)
# #randomize the target curvatrue
# np.random.seed(42)
# rho *= cscale*np.random.rand(*rho.shape)
# rho.shape

# colors = get_colors_trimesh(rho, plt.cm.jet)
# # colors[colors==0]=1

# # initialize mesh colored by rho
# mesh = trimesh.Trimesh(vertices=vert,faces=tria,
#                        face_colors=colors,
#                        face_attributes={'rho':rho})

# # mesh.show()

## DONT(later, after mvp): dev implementation using numpy-quaternion package
- it is difficult to do this with numpy-quaternion?
- Q: does quaternionic package handle matrix inverses?

In [ ]:
# import quaternionic
# import quaternion

In [ ]:
# # function V=spin(T,V,rho)
# # % input: T (3xN) and V (3xM) so that
# # %  trimesh(T',V(1,:),V(2,:),V(3,:)) plots the triangular mesh,
# # %  rho (1xN) values of conformal scaling
# # % output: V (3xM) vertices of transformed mesh
# #input: T,V,rho

# T = tria
# V = vert
# #input: T,V,rho
# #output: V or V_out,ome_out

# nT=T.shape[0]
# nV=V.shape[0]
# # plc=list(range(-3,1))
# # plc=np.array(list(range(-3,1)))
# #initialize virtual memory
# # E = scipy.sparse.csc_matrix((4*nV,4*nV), dtype=V.dtype)
# # E = scipy.sparse.csr_matrix((4*nV,4*nV), dtype=V.dtype)
# # E = scipy.sparse.bsr_matrix((4*nV,4*nV), dtype=V.dtype)
# # E = np.zeros((4*nV,4*nV), dtype=V.dtype)
# #Q: which is faster? csr or bsr or csc?
# # E = np.zeros((nV,nV), dtype=V.dtype)
# # E = np.zeros((nV,nV,4), dtype=np.quaternion)
# E = np.zeros((nV,nV), dtype=np.quaternion)

# edg=np.zeros(shape=(3,4))
# # pnty = np.zeros(4)
# pntz = np.zeros(4)
# # ini=np.zeros(shape=(3,4),dtype='int64')
# # ini=np.zeros(shape=(4,3),dtype='int64')
# edgy=np.zeros(shape=(3,4),dtype=V.dtype)
# E.dtype,edg.dtype,pnty,pntz,edgy.shape,edgy.dtype

In [ ]:
# for c1 in range(nT):
#     tri = T[c1]
#     pnt=V[tri]
#     #compute triangle area
#     A = 0.5*np.linalg.norm(np.cross(pnt[1]-pnt[0], pnt[2]-pnt[0]))
#     a = -0.25/A  # a=-1/(4*A)
#     #compute target curvature component
#     r = rho[c1]
#     b = r/6. #  b=rho(c1)/6;
#     # pnty[0]=A*r**2/9.
#     # c = jiH(pnty) #c=jiH([A*rho(c1)*rho(c1)/9 0 0 0]);
#     # c = np.quaternion(pnty)
#     c = np.quaternion(A*r**2/9.)
#     for c2 in range(3):
#         #Q: is this value right?
#         # V[tri[(c2+1)%3+1]] - V[tri[(c2+0)%3+1]]
#         pntz[1:] = V[tri[(c2+1)%3]] - V[tri[(c2+0)%3]]
#         edg[c2]  = pntz.copy()
#         # edg[c2]  = [0;V(:, tri(mod(c2+1,3)+1) )-V(:, tri(mod(c2+0,3)+1) )]
#     wooo = np.array([[
#         np.quaternion(*(a*jiH(edg[0])@edg[0]))+c, 
#         np.quaternion(*(a*jiH(edg[0])@edg[1]  + b*(edg[1]-edg[0])))+c, 
#         np.quaternion(*(a*jiH(edg[0])@edg[2]  + b*(edg[2]-edg[0])))+c], [
#         np.quaternion(*(a*jiH(edg[1])@edg[0]  + b*(edg[0]-edg[1])))+c, 
#         np.quaternion(*(a*jiH(edg[1])@edg[1]))+c,
#         np.quaternion(*(a*jiH(edg[1])@edg[2]  + b*(edg[2]-edg[1])))+c], [
#         np.quaternion(*(a*jiH(edg[2])@edg[0]  + b*(edg[0]-edg[2])))+c, 
#         np.quaternion(*(a*jiH(edg[2])@edg[1]  + b*(edg[1]-edg[2])))+c, 
#         np.quaternion(*(a*jiH(edg[2])@edg[2]))+c
#     ]])
#     E[np.ix_(tri,tri)] = E[np.ix_(tri,tri)] + wooo

In [ ]:
# q^{-1}=q'/(q*q')


In [ ]:
# (E.T*E)

In [ ]:
# quaternion.quaternion.

In [ ]:
# quaternion.quaternion_time_series.
# quaternion.means?

In [ ]:
# # quaternion.quaternion.inverse(E)
# (quaternion.as_spinor_array(np.quaternion(1,0,0,0)),
#     quaternion.as_spinor_array(np.quaternion(0,1,0,0)),
#     quaternion.as_spinor_array(np.quaternion(0,0,1,0)),
#     quaternion.as_spinor_array(np.quaternion(0,0,0,1)))

In [ ]:
#TODO: get the real case working...
#TODO: map float array to nV*2xnV*2 complex array and take the eigen decomposition.
# quaternion.as_float_array(E)

In [ ]:
# E.astype(np.complex64)

In [ ]:
# quaternion.as_spinor_array(E).shape

In [ ]:
# np.linalg.inv(quaternion.as_spinor_array(E))

In [ ]:
# (E/E).shape

In [ ]:
# #Q: does this work as a fast matrix inverse?
# q_identity = E/E
# boon=np.isnan(q_identity)
# q_identity[boon]=0*E[boon]
# q_identity[0][:3]

In [ ]:
# # boo_nan = np.isnan(q_identity[0][3:])
# boo_nan = np.isnan(q_identity[100])
# np.argwhere(boo_nan).all()

In [ ]:
# lam=np.zeros(nV,dtype=np.quaternion)+1
# # lam=zeros(4*nV,1);
# # lam(1:4:end)=1;  
# lam.shape

In [ ]:
# for c1 in range(11):
#     pass

# cnv=lam
# lam=lam/E
# lam.shape

In [ ]:
# boon=np.isnan(lam)
# boon.shape

In [ ]:
# q_identity[boon]=0*E[boon]

In [ ]:

# lam=lam/np.norm(lam)

In [ ]:
# for c1=1:11
#   cnv=lam;
#   lam=E\lam;
#   lam=lam/norm(lam);
# end

In [ ]:


# res=(E*lam)./lam;  
# fprintf('mean %e, var %e, delta %e\n',mean(res),var(res),norm(cnv-lam))


In [ ]:

# L  =sparse(4*nV,4*nV);
# ome=zeros(4*nV,1);
# for c1=1:nT
#   for c2=1:3
#     k0=T(mod(c2-1,3)+1,c1);
#     k1=T(mod(c2+0,3)+1,c1);
#     k2=T(mod(c2+1,3)+1,c1);
#     u1=V(:,k1)-V(:,k0);
#     u2=V(:,k2)-V(:,k0);
#     cta=dot(u1,u2) / norm( cross(u1,u2) );
#     h=jiH([cta*0.5 0 0 0]);
#     ini=[k1*4+plc  k2*4+plc];
#     L(ini,ini)=L(ini,ini)+[ h -h;-h h];
#     if k1>k2
#       k3=k1; k1=k2; k2=k3; % swap
#     end
#     lm1=jiH(lam(k1*4+plc));
#     lm2=jiH(lam(k2*4+plc));
#     edv=jiH([0;V(:,k2)-V(:,k1)]);
#     til=lm1'*edv*lm1/3 + lm1'*edv*lm2/6 + lm2'*edv*lm1/6 + lm2'*edv*lm2/3;
#     ome(k1*4+plc,1)=ome(k1*4+plc,1)-cta*til(:,1)/2;
#     ome(k2*4+plc,1)=ome(k2*4+plc,1)+cta*til(:,1)/2;
#   end
#   if ~mod(c1,500); fprintf('.'); end
# end
# fprintf('\n')

# ome=reshape(ome,[4 nV]);
# ome=ome-repmat(mean(ome,2),[1 nV]);
# ome=reshape(ome,[4*nV 1]);
# ome=L\ome;
# ome=reshape(ome,[4 nV]);
# ome=ome-repmat(mean(ome,2),[1 nV]);
# nrm=sum(ome.*ome,1);
# ome=ome/sqrt(max(nrm));
# V=ome(2:end,:);


In [ ]:
# # import numpy-quaternion
# !conda install -c conda-forge quaternion

In [ ]:
# # function h=jiH(pnt)
# # a=pnt(1); b=pnt(2); c=pnt(3); d=pnt(4);
# # h=[ a -b -c -d
# #     b  a -d  c
# #     c  d  a -b
# #     d -c  b  a];

# # @np.vectorize
# @njit
# def jiH(pnt):
#     """jiH castes a cartesion point, pnt, to a real quaternionic representation."""
#     a=pnt[0]; b=pnt[1]; c=pnt[2]; d=pnt[3];
#     h = np.array([
#         [a, -b, -c, -d],
#         [b,  a, -d,  c],
#         [c,  d,  a, -b],
#         [d, -c,  b,  a]])
#     return h

# # @njit
# # def jiH_vectorized(pnt_array):
# #     """jiH_vectorized complexifies an array of quaterions.
    
# #     Example Usage:
# # jiH_vectorized(a*jiH(edg[0]*edg[0]))
# #     """
# #     #h = np.array([jiH(x) for x in pnt_array]) #numba does not support this it seems...
# #     num = pnt_array.shape[0]
# #     h=np.empty(shape=(num,4,4))
# #     for i in range(num):
# #         h[i]=jiH(pnt_array[i])
# #     return h
    

In [ ]:
# #test jiH runs reasonably fast
# arr = np.reshape([0,1,2,3]*300,(300,4))
# a_lst=[]
# for pnt in arr:
#     a = jiH(pnt)
#     a_lst.append(a)
# #test jiH_vectorized runs reasonably fast
# # jiH_vectorized(np.concatenate(a_lst)).shape

# Step 1) represent the deformation as a matrix of real quaternions

In [ ]:
#heretim 
#def deform_mesh_conformally_real_simple(T,V,rho):
T = tria
V = vert
V.shape,V.dtype,rho.dtype

In [ ]:
tria.shape,vert.shape,rho.shape,np.max(rho)

In [ ]:
####################################
# Generate the Ensemble Matrix (E)
####################################
# function V=spin(T,V,rho)
# % input: T (3xN) and V (3xM) so that
# %  trimesh(T',V(1,:),V(2,:),V(3,:)) plots the triangular mesh,
# %  rho (1xN) values of conformal scaling
# % output: V (3xM) vertices of transformed mesh
#input: T,V,rho
#output: E
printing=True
nT=T.shape[0]
nV=V.shape[0]
# plc=list(range(-3,1))
plc=np.array(list(range(-3,1)))
step=0
num_steps=nT-1
update_printbar_every=int(np.around(nT/20))
#initialize virtual memory
# E = scipy.sparse.csc_matrix((4*nV,4*nV), dtype=V.dtype)
# E = scipy.sparse.csr_matrix((4*nV,4*nV), dtype=V.dtype)
# E = scipy.sparse.bsr_matrix((4*nV,4*nV), dtype=V.dtype)
E = np.zeros((4*nV,4*nV), dtype=V.dtype)
#Q: which is faster? csr or bsr or csc? I can caste to dense for solveh_banded later
edg=np.zeros(shape=(3,4))
pnty = np.zeros(4)
pntz = np.zeros(4)
ini=np.zeros(shape=(3,4),dtype=tria.dtype)
ini=np.zeros(shape=(3,4),dtype='int64')
# ini=np.zeros(shape=(4,3),dtype='int64')
# edgy=np.zeros(shape=(3,4),dtype=V.dtype)
# E.dtype,edg.dtype,pnty,pntz,edgy.shape,edgy.dtype

# for c1 in range(-1,nT):
# for c1 in range(1,nT):
for c1 in range(nT):
    tri = T[c1]
    pnt=V[tri]
    #pnt=V[tri].T #nope. 
    #compute triangle area
    A = 0.5*np.linalg.norm(np.cross(pnt[1]-pnt[0], pnt[2]-pnt[0]))
    a = -0.25/A  # a=-1/(4*A)
    #compute target curvature component
    r = rho[c1]
    b = r/6. #  b=rho[c1]/6;
    pnty[0]=A*r**2/9.
    c = jiH(pnty) #c=jiH([A*rho[c1]*rho[c1]/9 0 0 0]);

    for c2 in range(3):
#     for c2 in range(1,4):
        #Q: is this value right?
        # V[tri[(c2+1)%3+1]] - V[tri[(c2+0)%3+1]]
#         pntz[1:] = V[tri[(c2+1)%3]] - V[tri[(c2+0)%3]]
        pntz[1:] = V[tri[(c2+1)%3]] - V[tri[(c2+0)%3]]
        #pntz[1:] = -V[tri[(c2+1)%3]] + V[tri[(c2+0)%3]]
        #pntz[1:] = V[tri[(c2)%3]] - V[tri[(c2-1)%3]]
#         edg[(c2-2)%3]  = pntz.copy()
        edg[(c2-1)%3]  = pntz.copy()
#         edg[c2]  = pntz.copy()
#         edg[(c2+1)%3]  = pntz.copy()
        # edg[c2]  = [0;V(:, tri(mod(c2+1,3)+1) )-V(:, tri(mod(c2+0,3)+1) )]

    # signature of ini: 0ijk of triangle vertex 1,2,3
    # pseudo-signature of ini: vertex index as axis 0 & quaternion index as axis 1
    # signature of wooo: vertex index as axis 0,1 & quaternion index as axis 2,3
    #result: in matlab, 
    # E(ini,ini) indexes from the 0ijk fields of each triangle vertex 
    # to the 0ijk fields of each triangle vertex of that same triangle.
    ini[0] = tri[0]*4+plc
    ini[1] = tri[1]*4+plc
    ini[2] = tri[2]*4+plc 
#     ini[0] = (tri[0]+1)*4+plc 
#     ini[1] = (tri[1]+1)*4+plc
#     ini[2] = (tri[2]+1)*4+plc
    # ini=np.array([tri[0]*4+plc, tri[1]*4+plc, tri[2]*4+plc])
    # ini=[tri(1)*4+plc tri(2)*4+plc tri(3)*4+plc];
    # wo1 = np.array((jiH(a*jiH(edg[0])@edg[0])+c, 
    #     jiH(a*jiH(edg[0])@edg[1]  + b*(edg[1]-edg[0]))+c, 
    #     jiH(a*jiH(edg[0])@edg[2]  + b*(edg[2]-edg[0]))+c))
    # wo1.shape #(3, 4, 4)
    # wo2 = np.array((jiH(a*jiH(edg[1])@edg[0]  + b*(edg[0]-edg[1]))+c, 
    #     jiH(a*jiH(edg[1])@edg[1])+c,
    #     jiH(a*jiH(edg[1])@edg[2]  + b*(edg[2]-edg[1]))+c))
    # wo2.shape #(3, 4, 4)
    # wo3 = np.array((jiH(a*jiH(edg[2])@edg[0]  + b*(edg[0]-edg[2]))+c, 
    #     jiH(a*jiH(edg[2])@edg[1]  + b*(edg[1]-edg[2]))+c, 
    #     jiH(a*jiH(edg[2])@edg[2])+c))
    # wo3.shape #(3, 4, 4)
    # Q: does casting E to a sparse bsc matrix speed up any expensive linear algebra operations?
    wooo = np.array([[
        jiH(a*jiH(edg[0])@edg[0])+c, 
        jiH(a*jiH(edg[0])@edg[1]  + b*(edg[1]-edg[0]))+c, 
        jiH(a*jiH(edg[0])@edg[2]  + b*(edg[2]-edg[0]))+c], 
        [jiH(a*jiH(edg[1])@edg[0] + b*(edg[0]-edg[1]))+c, 
        jiH(a*jiH(edg[1])@edg[1])+c,
        jiH(a*jiH(edg[1])@edg[2]  + b*(edg[2]-edg[1]))+c], 
        [jiH(a*jiH(edg[2])@edg[0] + b*(edg[0]-edg[2]))+c, 
        jiH(a*jiH(edg[2])@edg[1]  + b*(edg[1]-edg[2]))+c, 
        jiH(a*jiH(edg[2])@edg[2])+c] 
    ])
#     wooo = np.array([[
#         jiH(a*jiH(edg[0])@edg[0])+c, 
#         jiH(a*jiH(edg[0])@edg[1]  + b*(edg[1]-edg[0]))+c, 
#         jiH(a*jiH(edg[0])@edg[2]  + b*(edg[2]-edg[0]))+c], 
#         [jiH(a*jiH(edg[1])@edg[0] + b*(edg[0]-edg[1]))+c, 
#         jiH(a*jiH(edg[1])@edg[1])+c,
#         jiH(a*jiH(edg[1])@edg[2]  + b*(edg[2]-edg[1]))+c], 
#         [jiH(a*jiH(edg[2])@edg[0] + b*(edg[0]-edg[2]))+c, 
#         jiH(a*jiH(edg[2])@edg[1]  + b*(edg[1]-edg[2]))+c, 
#         jiH(a*jiH(edg[2])@edg[2])+c] 
#     ])
#     ])
#Q: does transposing the output of jiH everywhere make the update to E agree with demo?
# #A: nope.?
#     wooo = np.array([[
#         jiH(a*jiH(edg[0]).T@edg[0])+c, 
#         jiH(a*jiH(edg[0]).T@edg[1]  + b*(edg[1]-edg[0]))+c, 
#         jiH(a*jiH(edg[0]).T@edg[2]  + b*(edg[2]-edg[0]))+c], 
#         [jiH(a*jiH(edg[1]).T@edg[0] + b*(edg[0]-edg[1]))+c, 
#         jiH(a*jiH(edg[1]).T@edg[1])+c,
#         jiH(a*jiH(edg[1]).T@edg[2]  + b*(edg[2]-edg[1]))+c], 
#         [jiH(a*jiH(edg[2]).T@edg[0] + b*(edg[0]-edg[2]))+c, 
#         jiH(a*jiH(edg[2]).T@edg[1]  + b*(edg[1]-edg[2]))+c, 
#         jiH(a*jiH(edg[2]).T@edg[2])+c] 
#     ])
    #woooo = wooo.reshape((12, 12))
#     #is ^this mucking it up?
# # #     wooo_= np.concatenate([wooo[2],wooo[1],wooo[0]],axis=1)
# # #     woooo = np.hstack([wooo_[2],wooo_[1],wooo_[0]])
# # #     wooo_= np.concatenate([wooo[2],wooo[0],wooo[1]],axis=1)
# # #     woooo = np.hstack([wooo_[2],wooo_[0],wooo_[1]])
#     wooo_= np.concatenate([wooo[1],wooo[2],wooo[0]],axis=1)
#     woooo = np.hstack([wooo_[1],wooo_[2],wooo_[0]])

#     #DONE: find the correct reshaping snippet
#     #reshape to square matrix local vertex representation
#     wooo_= np.concatenate([wooo[0],wooo[1],wooo[2]],axis=1)
#     woooo = np.hstack([wooo_[0],wooo_[1],wooo_[2]])
#     wooo_= np.concatenate([wooo[0],wooo[1],wooo[2]],axis=2)
#     woooo = np.vstack([wooo_[0],wooo_[1],wooo_[2]])
#     woooo=np.hstack([
#         np.concatenate([wooo[0,0], wooo[0,1], wooo[0,2]]),
#         np.concatenate([wooo[1,0], wooo[1,1], wooo[1,2]]),
#         np.concatenate([wooo[2,0], wooo[2,1], wooo[2,2]])])

#     woooo=np.hstack([
#         np.concatenate([wooo[0,0].T, wooo[0,1].T, wooo[0,2].T]),
#         np.concatenate([wooo[1,0].T, wooo[1,1].T, wooo[1,2].T]),
#         np.concatenate([wooo[2,0].T, wooo[2,1].T, wooo[2,2].T])]).T
#     woooo=np.hstack([
#         np.concatenate([wooo[0,0].T, wooo[1,0].T, wooo[2,0].T]),
#         np.concatenate([wooo[0,1].T, wooo[1,1].T, wooo[2,1].T]),
#         np.concatenate([wooo[0,2].T, wooo[1,2].T, wooo[2,2].T])]).T
    #reshape to square matrix local vertex representation
    woooo=np.hstack([
        np.concatenate([wooo[0,0], wooo[1,0], wooo[2,0]]),
        np.concatenate([wooo[0,1], wooo[1,1], wooo[2,1]]),
        np.concatenate([wooo[0,2], wooo[1,2], wooo[2,2]])]).T
    
#perturbing of ibid
#     woooo=np.hstack([
#         np.concatenate([wooo[0,0], wooo[1,0], wooo[2,1]]),
#         np.concatenate([wooo[0,1], wooo[1,1], wooo[2,0]]),
#         np.concatenate([wooo[1,2], wooo[0,2], wooo[2,2]])]).T

    
#     woooo=np.hstack([
#         np.concatenate([wooo[0,0].T, wooo[1,0].T, wooo[2,0].T]),
#         np.concatenate([wooo[0,1].T, wooo[1,1].T, wooo[2,1].T]),
#         np.concatenate([wooo[0,2].T, wooo[1,2].T, wooo[2,2].T])])

#     woooo=np.hstack([
#         np.concatenate([wooo[0,0], wooo[1,0], wooo[2,0]]),
#         np.concatenate([wooo[0,1], wooo[1,1], wooo[2,1]]),
#         np.concatenate([wooo[0,2], wooo[1,2], wooo[2,2]]),
#     ])
#obviously wrong
#     woooo=np.hstack([
#         np.concatenate([wooo[0,0].T, wooo[0,1].T, wooo[0,2].T]),
#         np.concatenate([wooo[1,0], wooo[1,1].T, wooo[1,2].T]),
#         np.concatenate([wooo[2,0], wooo[2,1], wooo[2,2].T])]).T

#     wooo_= np.concatenate([wooo[1],wooo[2],wooo[0]],axis=2)
#     woooo = np.vstack([wooo_[1],wooo_[2],wooo_[0]])
#     wooo_= np.concatenate([wooo[2],wooo[1],wooo[0]],axis=2)
#     woooo = np.vstack([wooo_[2],wooo_[1],wooo_[0]])
    #write to global vertex representation
    inii=ini.flatten()
    E[np.ix_(inii, inii)] += woooo
    #print progress bar
    if printing:
        step+=1
        if step%update_printbar_every==0:
            printProgressBar(step+1, num_steps, prefix = 'Progress:', suffix = 'Complete', length = 50)

In [ ]:
np.trace(E),E.dtype

- trace(E_identity): 56226.868111081465
- trace(E): 56100??

In [ ]:
print(np.around(E[np.ix_(inii, inii)],2))
# print(np.around(E[np.ix_(inii, inii)],3))
#DONE: try transposing the offdiagonal entries as they are written to E
# [[ 3.55  0.    0.    0.   -0.72 -0.07 -0.01  0.01 -0.33 -0.02 -0.06  0.02]
#  [ 0.    3.55  0.   -0.    0.07 -0.72  0.01  0.01  0.02 -0.33  0.02  0.06]
#  [-0.   -0.    3.55  0.    0.01 -0.01 -0.72 -0.07  0.06 -0.02 -0.33 -0.02]
#  [-0.    0.    0.    3.55 -0.01 -0.01  0.07 -0.72 
#DONE: make E(ini,ini) match the matlab demo exactly.


- ROOT CAUSE OF L DISAGREEMENT IDENTIFIED: E DISAGREES. 
- ROOT CAUSE RESOLVED.  it appears so, yes.
- DONE: verified all the entries of E agree in sign and magnitude

In [ ]:
# print(np.around(woooo,2))
# print(np.around(E[:9,:9],2))
print(np.around(E[1:6,1:6],2))
# print(np.around(E[1:10,1:10],2))

In [ ]:
print(np.around(E[-9:,-9:],2))

In [ ]:
# E_csc = scipy.sparse.csc_matrix(E)
# # #>3.5 minute runtime
# Einv_csc = scipy.sparse.linalg.inv(E_csc)
# Einv_ = Einv_csc.todense()

# E_bsr = scipy.sparse.bsr_matrix(E)
# # #3.5 min runtime
# Einv_bsr = scipy.sparse.linalg.inv(E_bsr)
##  SparseEfficiencyWarning: spsolve is more efficient when sparse b is in the CSC matrix format
#Einv_ = Einv_bsr.todense()

# #DONE: checked block inverse equals inverse to 13 significant digits
# np.max(Einv_-Einv)

In [ ]:
#plot E as a heatmap
# atol=1e-1
# atol=1e-3
atol=1e-5
# atol=1e-10
atol=1e-13
vmax=np.max((-np.min(E),np.max(E)))
vmax=-np.min(E)
# vmax=-np.min(E)/1000
boo=np.isclose(E,0.,atol=atol)
frac_nonzero = (~boo).sum()/np.sum(boo)
print(f"percent of matrix elements that are nonzero: {frac_nonzero:.4%}")

# vmax=0.01
# vmax=0.001
vmax=0.0001
figsize=(4,4)
# figsize=(7,7)
fig,ax=plt.subplots(figsize=(7,7))
ax.imshow(E,
    cmap='bwr',
# #     norm=None,
# #     aspect=None,
# #     interpolation=None,
# #     alpha=None,
    vmin=-vmax,
    vmax=vmax)
# format_plot
# format_plot(ax=ax,xlabel=r'n (1/cm$^2$)',ylabel=r'w (Hz/cm$^2$)',fontsize=fontsize)#,use_loglog=True)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.tick_params(top=True, right=True,direction='in',which='both')
plt.tight_layout()
plt.show()

In [ ]:
#test that E is symmetric #~6-15 second run time
assert np.isclose(E.T,E,5).all()
printing=True

In [ ]:
#done again below.
# #sparse soln: <20 second run time 
# if printing:
#     print(f"\nPerforming linear algebra solve...")
# lam,cnv = comp_groundstate_eigenvector_sparse(E)
# if printing:
#     print(f"linear solve complete!")
# res=(lam@E)/lam; 
# print(f'mean: {np.mean(res):e}, var:  {np.var(res):e}, delta: {np.linalg.norm(cnv-lam):e}')
# # mean: -1.825385e-04, var:  1.688050e-04, delta: 2.156203e+05
# # Q: how does lam look?
# # A: great! 
# #banded soln: ?? second run time.... does it crash the kernel?
# # mean: ??
# # Q: how does lam look?


In [ ]:
# # Eab = decomp_banded_matrix(E)
# E_sparse=scipy.sparse.csc_matrix(E,dtype=float)
# lam=np.zeros(4*nV)
# lam[::4]=1.
# b=lam
# # num_steps=11
# # for step in range(num_steps):
# #     cnv=lam
# #Q: what's the fastest way to compute x vs. x2 or x3
# x2 =scipy.sparse.linalg.lsqr(E_sparse,b)[0] #lam=lam/E
# # x3 =solveh_banded(Eab,b)#,lower=False)#,**kwargs) #lam =mldivide(lam,ab)
# #Q: what's the fastest way to compute x vs. x2 or x3
# # A: x2
# lam=x2.copy()
# # lam=x3.copy()

In [ ]:
use_inverse_method=False
if use_inverse_method:
    #Warning: 3 minute runtime for 11 epochs
    lam,cnv,Einv = comp_groundstate_eigenvector_inverse(E)
    res=(lam@E)/lam; 
    print(f'mean: {np.mean(res):e}, var:  {np.var(res):e}, delta: {np.linalg.norm(cnv-lam):e}')

In [ ]:
# Q: is Einv really needed?
# A: no.

## scratchwerk

In [ ]:
# #>4.5 minute runtime
# Einv=scipy.linalg.inv(E)

In [ ]:
# #~2 minute runtime
# Einv=np.linalg.inv(E)

In [ ]:
# #>7 minute runtime
# Einvp=np.linalg.pinv(E, rcond=1e-15, hermitian=True)

# #probably also >7 minute runtime
# Einvp=np.linalg.pinv(E, rcond=1e-15, hermitian=False)
# #heretim
# # Q: which matrix inversion methods work?
# # Q: of them, which are fastest?
# 16*11/60

In [ ]:
# np.mean(np.abs(Lp))

In [ ]:
# # ME=Einv_.copy()
# # ME=Einv.copy()
# ME=Lp.copy()
# # ME=Linv.copy() #<<<> not a useful quantity
# # vmax=1
# # vmax=0.00000000000001
# vmax=0.0001
# figsize=(4,4)
# # figsize=(7,7)
# fig,ax=plt.subplots(figsize=(7,7))
# print(f"{vmax=}")
# ax.imshow(ME,
#     cmap='bwr',
# # #     norm=None,
# # #     aspect=None,
# # #     interpolation=None,
# # #     alpha=None,
#     vmin=-vmax,
#     vmax=vmax)
# # format_plot
# ax.set_xticklabels([])
# ax.set_yticklabels([])
# ax.tick_params(top=True, right=True,direction='in',which='both')
# plt.tight_layout()
# plt.show()

In [ ]:
# Einv__ = scipy.linalg.pinvh(E) #<<<crashed the kernel...
# print(np.allclose(a, a @ B @ a))
# print(np.allclose(B, B @ a @ B))

In [ ]:
# 1

In [ ]:
# # Einv
# Einv=np.outer(y,y)
# Einv.shape

In [ ]:
# #2 min runtime
# #NOTE: it took 1.5 - 2 minutes to invert E in the real representation for the demo
# Einv = np.linalg.inv(E)

In [ ]:
# # Einv.shape
# #test the inverted matrix has only valid entries
# assert not np.isnan(Einv).any()
# assert not np.isinf(Einv).any()

In [ ]:
# #~2 minute runtime
# #test the inverse matrix works with left multiplication
# roo = np.abs( (Einv@E) - np.eye(E.shape[0]))
# print(roo[:3,:3])
# print(np.min(roo),np.max(roo)) #13 sig figs of agreement
# # [[3.78586051e-14 1.76031711e-14 1.90380781e-14]
# #  [1.00960100e-15 4.44089210e-16 8.99237282e-16]
# #  [1.76510237e-14 4.79217360e-17 4.44089210e-16]]
# # (0.0, 2.656980762521612e-13)

# #test the inverse matrix works with right multiplication
# roo = np.abs( (E@Einv) - np.eye(E.shape[0]))
# print(roo[:3,:3])
# print(np.min(roo),np.max(roo)) #14 sig figs of agreement

# # [[2.22044605e-16 8.06007486e-21 4.38189731e-19]
# #  [4.90931521e-18 2.22044605e-16 1.08420217e-18]
# #  [3.81132037e-18 2.16840434e-19 0.00000000e+00]]
# # (0.0, 1.4262223312566984e-14)

In [ ]:
# Q: should inverse be right multiplied or left multiplied everywhere?
# A: doesn't seem to matter up to a transpose.?

In [ ]:
# y=invh_banded(A=E,x=np.ones(E.shape[0]))
# y=invh_banded(A=E,x=np.zeros(E.shape[0]))

In [ ]:
# #>4 minute runtime
# wb,vb=scipy.linalg.eig_banded(ab)#, lower=False, eigvals_only=False, overwrite_a_band=False, select='a', select_range=None, max_ev=0, check_finite=True)

__Inverting E via LU Decomposition works, but it is not clearly going to be faster (it's $O(N^2)$)__

$$E = P L U$$

where $P$ is a permutation matrix, $L$ lower triangular with unit diagonal elements, and $U$ upper triangular.

In [ ]:
# #<4 minute runtime
# A=E.copy()
# # from scipy.linalg import lu
# p,l,u = scipy.linalg.lu(A, permute_l = False)

# l = np.dot(p,l) 

# l,u = scipy.linalg.lu(A, permute_l = True)

# l_inv = np.linalg.inv(l)

# u_inv = np.linalg.inv(u)

# A_inv = np.dot(u_inv,l_inv)

# # plt.plot(u_inv)
# # plt.plot(l_inv)

# # Q: can I use scipy.linalg.solveh_banded to furth speedup the leastsquare solve?  It's already <30 seconds...
#A: it appears <100ms per solve. let's gooo...! 
# Einv_=A_inv

# # np.dot(Einv_,E)
# eye=Einv_@E
# eye[:5,:5]

In [ ]:
# Fast method would be the Gauss-Jordan method

In [ ]:
# a = np.array([[1,2,0,0], [-1,2,1,0], [0,1,3,1], [0,0,1,2]])
# x = np.array([1,2,3,4])
# b = np.dot(a,x)
# ab = np.empty((3,4))
# ab[0,1:] = np.diag(a,1)
# ab[1,:] = np.diag(a,0)
# ab[2,:-1] = np.diag(a,-1) 
# yy =  scipy.linalg.solve_banded((1,1),ab,b)
# print (yy)

In [ ]:
# Einv
# Q: can i use lu decomp to estimate Einv?

In [ ]:
# Einv_=np.outer(y,y)#nope
# Einv_=np.outer(y.T,y)#nope
# Einv_=np.outer(y,y.T)#nope
# Einv_.shape,E.shape

In [ ]:
# plt.plot(y)

In [ ]:
# #heretim
# # vmax=0.01
# # vmax=0.001
# vmax=0.0001
# figsize=(4,4)
# # figsize=(7,7)
# fig,ax=plt.subplots(figsize=(7,7))
# ax.imshow(Einv,
#     cmap='bwr',
# # #     norm=None,
# # #     aspect=None,
# # #     interpolation=None,
# # #     alpha=None,
#     vmin=-vmax,
#     vmax=vmax)
# # format_plot
# # format_plot(ax=ax,xlabel=r'n (1/cm$^2$)',ylabel=r'w (Hz/cm$^2$)',fontsize=fontsize)#,use_loglog=True)
# ax.set_xticklabels([])
# ax.set_yticklabels([])
# ax.tick_params(top=True, right=True,direction='in',which='both')
# plt.tight_layout()
# plt.show()

In [ ]:
# #warning: these pythonic eigensolvers give different answers for the basic test case...
# # >>> don't assume hermiticity! return to the canned eigen pseudosolver!
# # >>> caution! i could be casting this into a real representation incorrectly!
# #warning: #>16 minutes run time
# #53-37 = >16 
# w, v = np.linalg.eig(E)
# w.shape,v.shape,np.min(w),np.max(w) 
# #for simplest unit sphere example:
# # ((264,),
# #  (264, 264),
# #  (-6.216318184123621-3.508736056987371j),
# #  (10.372886477259154+0j))

# #warning >7 minutes run time
# wh, vh = np.linalg.eigh(E)
# wh.shape,vh.shape,np.min(wh),np.max(wh)
# #for simplest unit sphere example:
# # ((264,), (264, 264), -13.124313869577566, 12.598788742101325)

In [ ]:
# E_bsr = scipy.sparse.bsr_matrix(E)#, dtype=V.dtype)
# E_bsr


# #4 min runtime
# Einv_bsr = scipy.sparse.linalg.inv(E_bsr)

# E_csc = scipy.sparse.csc_matrix(E)#, dtype=V.dtype)

# #>2 minute runtime
# Einv_csc = scipy.sparse.linalg.inv(E_csc)

In [ ]:
# # vals, vecs = scipy.sparse.linalg.eigs(E_bsr, k=6)
# vals, vecs = scipy.sparse.linalg.eigs(E_bsr, k=30)
# print(*vals)
# vals.shape, vecs.shape

In [ ]:
# E_bsr = scipy.sparse.bsr_matrix(E)#, dtype=V.dtype)
# E_bsr
# # vals, vecs = scipy.sparse.linalg.eigs(E_bsr, k=6)
# vals, vecs = scipy.sparse.linalg.eigs(E_bsr, k=30)
# print(*vals)
# vals.shape, vecs.shape

In [ ]:
# print(np.min(E),np.max(E))
# print(np.min(Einv),np.max(Einv))
# # -0.7250259143115632 3.5552214831079576
# # -1.3519086938558618 2.7442374749979765

In [ ]:
#input: Einv,V,T
#output: ome

In [ ]:
# #input: Einv
# #output: lam approximating the groundstate eigenvector
# lam=np.zeros(4*nV)
# # lam[1::4]=1.
# # lam[-1::4]=1.
# # lam[::4]=1.
# lam[3::4]=1.
# lam/=np.linalg.norm(lam)
# # lam+=1.
# #Q: is ^this right?
# #Q is this better? lam[3::4]=1.
# ## lam(1:4:end)=1;  
# print(lam.shape)
# # for c1 in range(11):
# # for c1 in range(111):
# # for c1 in range(11):
# # for c1 in range(12):
# for c1 in range(11):
#     cnv=lam
#     #lam=lam/E
#     lam =lam@Einv
# #     lam =Einv@lam
#     #Q: left multiply instead or right multiply?
#     #heretim
#     lam/=np.linalg.norm(lam);
# print(lam.shape)
# # np.min(lam),np.max(lam)
# # res=(E@lam)/lam; 
# res=(lam@E)/lam; 
# # print(f'mean: {np.mean(res):.4f}, var:  {np.var(res):.4f}, delta: {np.linalg.norm(cnv-lam):.4f}')
# print(f'mean: {np.mean(res):e}, var:  {np.var(res):e}, delta: {np.linalg.norm(cnv-lam):e}')
# # fprintf('mean %e, var %e, delta %e\n',mean(res),var(res),norm(cnv-lam))
# # mean: 0.0002, var:  0.0000, delta: 0.0000
# #reproduced here (right divide):
# # mean: 2.904111e-05, var:  7.868836e-25, delta: 8.198954e-14
# #reproduced here (left divide):
# # mean: 2.904111e-05, var:  1.888099e-27, delta: 8.550921e-14
# #expected from matlab:
# # mean 9.446027e-05, var 5.570631e-28, delta 1.018211e-12
# # mean: 6.156186e-04, var:  6.571099e-04, delta: 4.380033e-01
# # mean: 6.670979e-18, var:  5.166021e-26, delta: 1.526158e-16
# # mean: -2.961917e-04, var:  5.994400e-24, delta: 2.000000e+00
# # mean: 9.446027e-05, var:  3.590215e-26, delta: 5.604947e-14

# #DONT: #adjust indexing  
# # lam = np.roll(lam,1) #<<< looks wrong
# # lam = np.roll(lam,-1) #<<< looks nearly plausible
# # #heretim

# # lam_=list(lam[:10])
# lam_=list(lam[:6])
# # lam_=list(lam[:6])
# # lam_=list(np.roll(lam[::-1],1)[:6])
# # lam_=list(np.roll(lam,3)[:6])
# print(f"\nlam_:")
# print(f"{lam_}")
# res.shape,res.dtype

In [ ]:
# L,ome=map_lam_real(lam,V,T,printing=True)#,**kwargs)
# print(f"\n{L.shape=}, {ome.shape=}")
# print(f"{np.min(L)=},{np.max(L)=}")
# print(f"{np.min(ome)=},{np.max(ome)=}")
# # print(f"{np.around(L[:10,:10],2)=}")
# # print(f"{ome[:6]=}")
# Lp=np.roll(np.roll(L,3,axis=0),3,axis=1)
# omep=np.roll(ome,3)

# #center input
# ome = omep.reshape((nV,4))
# ome = ome - np.broadcast_to(np.mean(ome,axis=0), shape=(nV,4), subok=False)
# ome = ome.flatten()
# A = csc_matrix(Lp, dtype=float) #<<< looks fastest for demo rho
# # A = scipy.sparse.bsr_matrix(Lp, dtype=float)
# # A = scipy.sparse.csr_matrix(Lp, dtype=float)
# #solve the linear system in the least square
# b = np.array(ome, dtype=float)
# x, istop, itn, normr = lsqr(A, b)[:4]
# print(f"{istop=}")

In [ ]:
# #Q: what's the fastest way to compute x vs. x2 or x3
# #A: it appears Lp_sparse is fastest
# Lp_sparse=scipy.sparse.csc_matrix(Lp,dtype=float)
# x2 =scipy.sparse.linalg.lsqr(Lp_sparse,b)[0] #lam=lam/E

In [ ]:
# Lpab = decomp_banded_matrix(Lp)
# Lp_sparse=scipy.sparse.csc_matrix(Lp,dtype=float)
# # num_steps=11
# # for step in range(num_steps):
# #     cnv=lam


In [ ]:
# Eab = decomp_banded_matrix(E)


In [ ]:
# np.min(b),np.max(b)

In [ ]:
# np.min(lam),np.max(lam)

In [ ]:

# # LinAlgError: 1th leading minor not positive definite 
# x3 =solveh_banded(Lpab,b)#,lower=False)#,**kwargs) #lam =mldivide(lam,ab)
# x3 =solveh_banded(Eab,b)#,lower=False)#,**kwargs) #lam =mldivide(lam,ab)
#  LinAlgError: 1th leading minor not positive definite 
# x3 =solveh_banded(Eab,lam)#,lower=False)#,**kwargs) #lam =mldivide(lam,ab)
#  LinAlgError: 1th leading minor not positive definite 

In [ ]:
#Q: do lm1,lm2at the end of the for loop match?
#A: yes!


In [ ]:
# # Einv_=scipy.sparse.inv(E)
# E_sparse=scipy.sparse.csc_matrix(E,dtype=float)

In [ ]:
# #5 minutes using the float64
# Einv_sparse=scipy.sparse.linalg.inv(E_sparse)
# # scipy.sparse.linalg.isolve

In [ ]:
# Einv_sparse.dtype

In [ ]:
# E_bsr=scipy.sparse.bsr_matrix(E,dtype=float)

In [ ]:
# #6 minute runtime
# Einv_sparse2=scipy.sparse.linalg.inv(E_bsr)

In [ ]:
# Einv_sparse2.dtype

In [ ]:
#TODO: wrap map from V,T to E,lam,Einv # run time <3 minutes bc matrix inversion
#TODO: wrap map from V,T,E,lam,?? to V_out # run time <10 seconds
#TODO: wrap map from V,T,E,lam to V_out

In [ ]:
#WARNING: numpy slow here 
# # np.linalg.lstsq?
# np.linalg.lstsq(a, b, rcond='warn')
# Docstring:
# Return the least-squares solution to a linear matrix equation.

## DONE: visualize the least-square solution 

In [ ]:
# np.max(lam),np.linalg.norm(lam)
# # (0.0191443592787548, 1.0)

# L,ome=map_lam_real(lam,V,T,printing=True)#,**kwargs)
# print(f"\n{L.shape=}, {ome.shape=}")
# print(f"{np.min(L)=},{np.max(L)=}")
# print(f"{np.min(ome)=},{np.max(ome)=}")
# # print(f"{np.around(L[:10,:10],2)=}")
# # print(f"{ome[:6]=}")

# Lp=np.roll(np.roll(L,3,axis=0),3,axis=1)
# omep=np.roll(ome,3)

In [ ]:
# print(f"Q: do these agree with the matlab readout?")
# print(f"A: for L, yes! for ome, no...")
# # Lp=np.roll(L,(3,-3))
# # Lp=np.roll(L,(-3,3))
# # Lp=np.roll(L,-3)
# # Lstart=np.around(L[:9,:9],3)
# Lstart=np.around(Lp[:10,:10],3)
# omestart=list(omep[:6]*1e5)
# print(f"Lstart:")
# print(f"{Lstart}")
# print(f"\nome_start:")
# print(f"{omestart}")
# # [-1.287227454169968e-17, -0.014770930578963743, 0.056678765468954485, -0.0033788113455863788, 5.4603064682953105e-18, -0.01652640834011626]
# # [-1.287227454169968e-17, 0.014770930578963785, -0.05667876546895455, 0.0033788113455863996, 5.4603064682953105e-18, 0.01652640834011626]


In [ ]:
# list(omep[:10])

In [ ]:
# FIXED: - it looks like some vector components are switched in the solution start, ome_start:
#             - i:j, j:k, k:i 
#             - x:y, y:z, z:x

In [ ]:
# # L_=np.around(Lp[:9,:9],3)
# L_=np.around(Lp[-9:,-9:],3)
# ome_=list(omep[:6]*1e5)
# print(f"L_:")
# print(f"{L_}")
# print(f"\nome_:")
# print(f"{ome_}")

# #DONE: checked L agrees at its head and tail
# #PROBLEM IDENTIFIED #the z component is wrong... in ome

In [ ]:
# #plot the computed groundstate
# plt.plot(lam[::4])
# plt.plot(lam[1::4])
# plt.plot(lam[2::4])
# plt.plot(lam[3::4])

In [ ]:
#>12 minute run time (might overestimate bc swap_ram=95%
# # Q: can I get a given state of eigenspectrum this way?
# #get the first two eigenvectors
# Ab = decomp_banded_matrix(A=E)
# w, v = scipy.linalg.eig_banded(Ab, lower=True, select='v', select_range=[0,1])
# lamv=v[0]
# lamv.shape


In [ ]:
#DONE: dev/test mldivide_banded
#DONE: replace all matrix inversions with mldivide_banded iterated 11 times
#DONE: test equivalence with incumbant solution. it's slower...

ROOT CAUSE IDENTIFIED:
- there are differences in L
- there are differences in ome
- Q: can i shift L,ome in a way that make the difference go away? 
- A: yes.  there are still differences in ome, however...

# Step 2) visualize the least-square approximate to the desired deformation

In [ ]:
printing=True
#sparse soln: <20 second run time 
print(f"estimating groundstate eigenvector (estimated run time <20 seconds)... ")
#sparse soln: <20 second run time 
if printing:
    print(f"\nPerforming linear algebra solve...")
lam,cnv = comp_groundstate_eigenvector_sparse(E)
if printing:
    print(f"linear solve complete!")
res=(lam@E)/lam; 
print(f'mean: {np.mean(res):e}, var:  {np.var(res):e}, delta: {np.linalg.norm(cnv-lam):e}')

In [ ]:
# from scipy.sparse import csc_matrix
# from scipy.sparse.linalg import lsqr
#plot the computed groundstate
plt.plot(lam[::4])
plt.plot(lam[1::4])
plt.plot(lam[2::4])
plt.plot(lam[3::4])
print(np.max(lam),np.linalg.norm(lam))
plt.show()

In [ ]:
#heretim
#DONE: wrap from here to V_out
#DONE: wrap all this in a routine
#DONE: make it as fast as possible (first)
x,Lp,ome = map_real_lam_to_vertices(lam,V,T,printing=True,use_return_Lp=True)#,**kwargs)

In [ ]:
lam0=lam.copy() #initial (trivial) state
Lp0=Lp.copy()
ome0=ome.copy()
# lam1=lam.copy()  #final state
# Lp1=Lp.copy()
# ome1=ome.copy()

In [ ]:
lam0.shape,lam1.shape

In [ ]:
#save lam0,lam1 as numpy arrays
lam_dir = '/Users/timothytyree/Documents/meshes/res/rho_grnd_state.npz'
np.savez_compressed(lam_dir, rho=rho, lam0=lam0, lam1=lam1, tria=tria)#, vert=vert)
assert os.path.exists(lam_dir)

In [ ]:
# dict_arr=dict(
#     lam0
#     Lp0=Lp0,
#     ome0=ome0,
#     Lp1
# )
# ome0.shape

In [ ]:
# #test lam0 is not to be zero to machine precision
# assert np.isclose(lam0,0).sum()==np.isclose(lam0,0).shape[0]

In [ ]:
om=x.copy()
#center result
om = om.reshape((nV,4))
om = om - np.broadcast_to(np.mean(om,axis=0), shape=(nV,4), subok=False)
#normalize output mesh
nrm = np.sum(om*om,axis=0)
# nrm=sum(ome.*ome,1);
ome_out=om/np.sqrt(np.max(nrm))
# ome_out=om#/np.sqrt(np.max(nrm))
# ome=ome/sqrt(max(nrm));
V_out = ome_out[:,1:]
scalar_out = ome_out[:,0]
# V=ome(2:end,:);
print(f"\nQ: is the smallest value in the first index?\m")
print(f"*** Norm: {list(nrm)} ***")
# np.min(scalar_out),np.max(scalar_out)
assert vert.shape==V_out.shape
assert not np.isnan(V_out).any()
# np.max(np.abs(V_out)),np.mean(np.abs(V_out))
V_out.dtype,V_out.shape

In [ ]:
#normalize output mesh
nrm = np.sum(ome_out*ome_out,axis=0)
# # nrm=sum(ome.*ome,1);
# ome_out=om/np.sqrt(np.max(nrm))
# # ome_out=om#/np.sqrt(np.max(nrm))
# # ome=ome/sqrt(max(nrm));
# V_out = ome_out[:,1:]
# scalar_out = ome_out[:,0]
# # V=ome(2:end,:);
print(f"\nQ: is the smallest value in the first index?\m")
print(f"*** Norm: {list(nrm)} ***")
# V_out = om[:,1:]
# scalar_out = om[:,0]
# np.min(scalar_out),np.max(scalar_out)
# np.max(np.abs(V_out)),np.mean(np.abs(V_out))

In [ ]:
#visualize the output mesh
# initialize mesh colored by rho
mesh_out = trimesh.Trimesh(vertices=V_out,faces=tria,
                       face_colors=colors,
                       face_attributes={'rho':rho})
mesh_out.vertices-=mesh_out.center_mass
#normalize output mesh
# mesh_out.vertices/=np.sign(mesh_out.volume)*np.abs(mesh_out.volume)**0.33
mesh_out.vertices*=np.sign(mesh_out.volume) #account for double cover edge case from quaternionic representation
mesh_out.vertices/=np.abs(mesh_out.bounding_sphere.volume)**0.33
nrm = np.sum(mesh_out.vertices*mesh_out.vertices,axis=0)

# mesh_out.vertices/=np.abs(mesh_out.bounding_sphere.volume)**0.33

# #(unecessary?) repair mesh
# trimesh.repair.fix_inversion(mesh_out)
# trimesh.repair.fix_normals(mesh_out)
# trimesh.repair.fix_winding(mesh_out)

#print mesh bluf
print(f"{mesh_out.is_watertight=}")
print(f"{mesh_out.volume=}")
print(f"{(mesh_out.volume/mesh_out.bounding_sphere.volume)=} (volume relative to smallest bounding sphere)")
print(f"{mesh_out.center_mass=}")
print(f"rms norm of mesh vertices: {list(np.sqrt(nrm))} pixels")
print(f"{mesh_out.is_watertight=} (is the current mesh watertight?)")
print(f"{mesh_out.euler_number=} (what's the euler number for the mesh?)")
print(f"volume relative to smallest bounding sphere: {(mesh_out.volume/mesh_out.bounding_sphere.volume)=}")
beep(3)

In [ ]:
mesh_out.show()

In [ ]:
volume_initial = mesh.volume#=4.182994798204928
volume_final = mesh_out.volume#=4.182994798204928
print(f"{volume_initial=:.4f} voxels, {volume_final=:.4f} voxels")
print(f"{mesh.bounding_sphere.volume=:.4f} voxels, {mesh_out.bounding_sphere.volume=:.4f} voxels")
print(f"{volume_initial/mesh.bounding_sphere.volume=:.4f} voxels, {volume_final/mesh_out.bounding_sphere.volume=:.4f} voxels")
#compute the percent change in relative volume
relative_volume_initial = volume_initial/mesh.bounding_sphere.volume
change_in_relative_volume = volume_final/mesh_out.bounding_sphere.volume - volume_initial/mesh.bounding_sphere.volume
relative_change_in_relative_volume = change_in_relative_volume / relative_volume_initial
print(f"percent change in relative volume: {relative_change_in_relative_volume:.4%}")

In [ ]:
#plot the computed groundstate
# plt.plot(x[::4])
# plt.plot(x[1::4])
# plt.plot(x[2::4])
# plt.plot(x[3::4])
# plt.show()
#plot the computed vertex representation of the transformation
plt.plot(om[:,0])
plt.plot(om[:,1])
plt.plot(om[:,2])
plt.plot(om[:,3])
plt.show()

# TODO(later): dev interpolating map video






## TODO: Simplest case) linear interpolation
then view it and decide if anything more complicated is really necessary 

In [ ]:
lam_dir = '/Users/timothytyree/Documents/meshes/res/rho_grnd_state.npz'
arr=np.load(lam_dir)
# print(arr.keys())
lam0=arr['lam0']
lam1=arr['lam1']
tria=arr['tria']
rho=arr['rho']
T=tria
V=vert

In [ ]:
#DONE: verify eye is the identity
#DONE: caste eye to quaternion dtype as q0
#DONE: caste Lp to quaternion dtype as q1
#DONE: use quaternion dtype's slerp interpolation method
#DONE: caste interpolated q back to same representation as L
# Q: does ^this need to be done for ome, too?
# A: it looks like i can just slerp x0 to x1
#DONT(?): do that <30 second linear solve to get om

In [ ]:
import quaternion

def map_to_quaternion_matrix(Lp):
    #map from Lp to q1
    wm=Lp[::4,::4]
    wx=Lp[1::4,1::4]
    wy=Lp[2::4,2::4]
    wz=Lp[3::4,3::4]
    W = np.stack([wm,wx,wy,wz],axis=2)
    q1 = quaternion.from_float_array(W)
    return q1

def map_from_quaternion_matrix(qm):
    WT = quaternion.as_float_array(qm).T
    nV = qm.shape[0]
    Lq = np.zeros((4*nV,4*nV))
    Lq[::4,::4]   = WT[0]
    Lq[1::4,1::4] = WT[1]
    Lq[2::4,2::4] = WT[2]
    Lq[3::4,3::4] = WT[3]
    return Lq

#DONE: dev similar maps for vectors of quaternions
def map_to_quaternion_vector(v):
    #map from Lp to q1
    wm=v[::4]
    wx=v[1::4]
    wy=v[2::4]
    wz=v[3::4]
    W = np.stack([wm,wx,wy,wz],axis=1)
    qv = quaternion.from_float_array(W)
    return qv

def map_from_quaternion_vector(qv):
    WT = quaternion.as_float_array(qv).T
    nV = qv.shape[0]
    wv = np.zeros(4*nV)
    wv[::4]   = WT[0]
    wv[1::4] = WT[1]
    wv[2::4] = WT[2]
    wv[3::4] = WT[3]
    return wv

In [ ]:
def map_quaternions_to_trimesh(qstar,tria,colors,face_attributes,**kwargs):
    """map_quaternions_to_trimesh maps qstar to mesh_current.
    qstar is a 1D numpy.array of quaternion dtype with a length of the number of vertices.
    additional kwargs are passed to trimesh.Trimesh.
    
    Example Usage:
face_attributes={'rho':rho}
mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)
    """
    om=quaternion.as_float_array(qstar).copy()
    nV=om.shape[0]
    #center result
    om = om.reshape((nV,4))
    om = om - np.broadcast_to(np.mean(om,axis=0), shape=(nV,4), subok=False)
    #normalize output mesh
    nrm = np.sum(om*om,axis=0)
    ome_current=om/np.sqrt(np.max(nrm))
    V_current = ome_current[:,1:]
    # initialize mesh colored by face_attributes
    mesh_current = trimesh.Trimesh(vertices=V_current,faces=tria,
                           face_colors=colors,
                           face_attributes=face_attributes,**kwargs)
    mesh_current.vertices-=mesh_current.center_mass
    #normalize output mesh
    mesh_current.vertices*=np.sign(mesh_current.volume) #accounts for double cover edge case from quaternionic representation
    mesh_current.vertices/=np.abs(mesh_current.bounding_sphere.volume)**(1./3.) #without bounding sphere
    # mesh_current.vertices/=np.sign(mesh_current.volume)*np.abs(mesh_current.volume)**0.33 #without bounding sphere
    return mesh_current


def update_quaternions_to_trimesh_vertices(qstar,mesh_current,
                                           center_result=True,
                                           normalize_result=True,
                                           center_result_again=True,
                                           normalize_by_bounding_sphere=True,
                                           **kwargs):
    """update_quaternions_to_trimesh_vertices updates the vertices of mesh_current with qstar.
    qstar is a 1D numpy.array of quaternion dtype with a length of the number of vertices.
    
    Example Usage:
update_quaternions_to_trimesh_vertices(qstar,mesh_current,center_result=True,
                                           normalize_result=True,center_result_again=True,
                                           normalize_by_bounding_sphere=True)#,**kwargs)
    """
    om=quaternion.as_float_array(qstar).copy()
    if center_result:
        nV=om.shape[0]
        #center result
        om = om.reshape((nV,4))
        om = om - np.broadcast_to(np.mean(om,axis=0), shape=(nV,4), subok=False)
    if normalize_result:
        #normalize output mesh
        nrm = np.sum(om*om,axis=0)
        ome_current=om/np.sqrt(np.max(nrm))
    else:
        ome_current=om
    if center_result_again:
        mesh_current.vertices =ome_current[:,1:]
        mesh_current.vertices-=mesh_current.center_mass
    if normalize_by_bounding_sphere:
        #normalize output mesh
        mesh_current.vertices*=np.sign(mesh_current.volume) #accounts for double cover edge case from quaternionic representation
        mesh_current.vertices/=np.abs(mesh_current.bounding_sphere.volume)**(1./3.) #without bounding sphere
        # mesh_current.vertices/=np.sign(mesh_current.volume)*np.abs(mesh_current.volume)**0.33 #without bounding sphere
    return mesh_current


In [ ]:
x,Lp,ome = map_real_lam_to_vertices(lam,V,T,printing=True,use_return_Lp=True)#,**kwargs)
Lp.shape,x.shape,ome.shape

In [ ]:
v = ome.copy()
v = x.copy()

In [ ]:
#map from Lp to q1
q1 = map_to_quaternion_matrix(Lp)
#map from q1 to Lq
Lq = map_from_quaternion_matrix(q1)
#test equivalence between Lp and Lq
assert (Lp==Lq).all()

qv = map_to_quaternion_vector(v)
wv = map_from_quaternion_vector(qv)
#test equivalence between v and wv
assert (wv==v).all()

## DONE: dev slerp
Q: can I use the quaternionic vector solutions directly?

In [ ]:
fig,axs=plt.subplots(ncols=2,figsize=(12,4))
# #plot the initial/final states
plt.sca(axs[0])
plt.plot(lam0[::4])
plt.plot(lam0[1::4])
plt.plot(lam0[2::4])
plt.plot(lam0[3::4])
plt.sca(axs[1])
plt.plot(lam1[::4])
plt.plot(lam1[1::4])
plt.plot(lam1[2::4])
plt.plot(lam1[3::4])
plt.show()

In [ ]:
#generate initial state
x0,Lp0,ome0 = map_real_lam_to_vertices(lam0,V,T,printing=True,use_return_Lp=True)#,**kwargs)
q0 = map_to_quaternion_vector(x0)

In [ ]:
#generate final state
x1,Lp,ome = map_real_lam_to_vertices(lam1,V,T,printing=True,use_return_Lp=True)#,**kwargs)
q1 = map_to_quaternion_vector(x1)


In [ ]:
# # Calculate slerp from arrays of (q_1, q_2, tau)
#slerp_vectorized signature:
# np.slerp_vectorized(x1, x2, x3, /, out=None, *, where=True, casting='same_kind', order='K', dtype=None, subok=True[, signature, extobj])
#squad_vectorized signature takes a list of states, potentially from ibid
# np.squad_vectorized

In [ ]:
# # Calculate slerp from arrays of (q_1, q_2, tau)
# #DONE: test slerp reproduces the endpoints of interpolation for a given element
# q = np.slerp_vectorized(q00,q11.conjugate(),0)
# assert q==q00
# q = np.slerp_vectorized(q00,q11.conjugate(),1)
# assert np.isclose(q,q11)

In [ ]:
#test slerp reproduces the endpoints of interpolation for whole arrays of quaternions
qstar = np.slerp_vectorized(q0,q1.conjugate(),0)
assert (quaternion.as_float_array(qstar-q0)==0.).all() #<<< passed with demo
qstar = np.slerp_vectorized(q0,q1.conjugate(),1)
assert np.isclose(quaternion.as_float_array(qstar-q1),0.,atol=1e-6).all() #<<< passed with demo
# assert np.isclose(quaternion.as_float_array(qstar-q1),0.,atol=1e-7).all() #<<< failed with demo

# qstar==q00
# np.isclose(qstar,q0)
# assert q==q00
# q = -np.slerp_vectorized(q00,q11,1)
# assert np.isclose(q,q11)
qstar.shape,q1.shape

In [ ]:
q1c=q1.conjugate()
N=q1.shape[0]
mesh_current=None

In [ ]:
# iterate over tau

In [ ]:
#TODO: figure out how to make this continuous at tau=0
#NOTE: there's a flip of sign at tau=0...
#NOTE: who cares if the other end has a rotation flip?
qstar_lst=[]
dict_qstar_lst=[]
N=q1.shape[0]
mesh_current=None
for tau in tau_values:
    #map to vertex representation
    qstar = np.slerp_vectorized(q0,q1c,tau) #continuous at 0
    face_attributes={'rho':rho} #optionally, update local face attributes <<< FVM access point! (useful!) 
    if mesh_current is None:
        mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)
    else:
        update_quaternions_to_trimesh_vertices(qstar,mesh_current,center_result=True,
                                           normalize_result=True,center_result_again=True,
                                           normalize_by_bounding_sphere=True)#,**kwargs)
    V=float(mesh_current.volume)
#     #mesh_current.vertices = mesh_current.vertices.astype(dtype='float32') ##<<< any effect on curvature runtime?

    #isovolemic constraint
    vscale = float(V0/V)**(1./3.)
    mesh_current.vertices *= vscale
    qstar*= vscale #<<< any effect on mean displacement traces?
#     qstar[::4]*= float(V0/V) #<<< any effect on mean displacement traces?
#     qstar*= float(V0/V) #<<< any effect on mean displacement traces?
    q1scaled = q1 * vscale
    D0=quaternion.as_float_array(qstar-q0)
    D1=quaternion.as_float_array(qstar-q1scaled)
    D0*=D0.conjugate()
    D1*=D1.conjugate()
    delta0 = float(D0.mean())
    delta1 = float(D1.mean())
    Delta_delta0 = 1.96*D0.std()/np.sqrt(N) #95% confidence interval
    Delta_delta1 = 1.96*D1.std()/np.sqrt(N) #95% confidence interval
    dict_qstar=dict(
        tau=tau,
        delta0=delta0,
        delta1=2*delta1,
        Delta_delta0=Delta_delta0,
        Delta_delta1=Delta_delta1,
        V=
    )
    #record
    dict_qstar_lst.append(dict_qstar)
    qstar_lst.append(qstar)
    
    #print progress bar
    if printing:
        step+=1
        if step%update_printbar_every==0:
            printProgressBar(step+1, num_steps, prefix = 'Progress:', suffix = 'Complete', length = 50)
    
df_qstar=pd.DataFrame(dict_qstar_lst)
del dict_qstar_lst
df_qstar.shape,qstar.shape

In [ ]:
delta1,delta0

In [ ]:
#Q: does q1.conjugate() remove the flip disagreement at the end?
#A: not anymore. ;)
fontsize=14
fig,ax = plt.subplots(figsize=(6,4))
df_qstar.plot(x='tau',y='delta0',label=r'$\delta_0$',ax=ax)
df_qstar.plot(x='tau',y='delta1',label=r'$\delta_1$',ax=ax)
AddLegend(ax=ax,fontsize=fontsize)
format_plot(ax=ax,xlabel=r'$\tau$',ylabel=r'Mean Displacement',fontsize=fontsize)
# plt.yscale('log')
plt.show()

In [ ]:
#TODO: dev map from qstar to vertex representation
# input: qstar
# output: mesh
beep(4)

In [ ]:
# tau_values = np.arange(0,1+1e-2,1e-2)
tau_values = df_qstar['tau'].values
tau_values.shape

In [ ]:
# comp_trimesh_curvature(mesh_current)

In [ ]:
#TODO: estimate mesh volume versus tau
#TODO: estimate mesh mean curvature, rhobar, versus tau
# use_compute_curvature=False
V_lst=[]
rhobar_lst=[]
# for i,tau in enumerate(df_qstar.tau.values):
step=0
num_steps=tau_values.shape[0]
print(f"{num_steps=}")
update_printbar_every=2
# tau_values = np.arange(0,1+1e-3,1e-3):
V0=float(mesh.volume)
# mesh_current=None
for tau in tau_values:
    #map to vertex representation
    qstar = np.slerp_vectorized(q0,q1c,tau) #continuous at 0
    face_attributes={'rho':rho} #optionally, update local face attributes <<< FVM access point! (useful!) 
    if mesh_current is None:
        mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)
    else:
        update_quaternions_to_trimesh_vertices(qstar,mesh_current,center_result=True,
                                           normalize_result=True,center_result_again=True,
                                           normalize_by_bounding_sphere=True)#,**kwargs)
    V=float(mesh_current.volume)
    mesh_current.vertices = mesh_current.vertices.astype(dtype='float16') ##<<< any effect on curvature runtime? 18 minutes runtime...
    #mesh_current.vertices = mesh_current.vertices.astype(dtype='float32') ##<<< any effect on curvature runtime?
    
    #isovolemic constraint
    vscale = float(V0/V)**(1./3.)
    mesh_current.vertices *= vscale
    qstar*= vscale #<<< any effect on mean displacement traces?
    
#     #map to vertex representation
#     qstar = np.slerp_vectorized(q0,q1c,tau) #continuous at 0
#     face_attributes={'rho':rho}
#     mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)
#     V=float(mesh_current.volume)
#     #mesh_current.vertices = mesh_current.vertices.astype(dtype='float32') #saved 4 minutes off 100 iterations
#     mesh_current.vertices = mesh_current.vertices.astype(dtype='float16') #saved 4 minutes off 100 iterations
#     #isovolemic constraint
#     mesh_current.vertices *= float(V0/V)**(1./3.)
    #compute curvature (expensive!)
    rho_ = comp_trimesh_curvature(mesh_current)#,radius=1./(2.*np.pi))
    rhobar=float(np.mean(rho_))
    #record
    V_lst.append(V)
    rhobar_lst.append(rhobar)
    #print progress bar
    if printing:
        step+=1
        if step%update_printbar_every==0:
            printProgressBar(step+1, num_steps, prefix = 'Progress:', suffix = 'Complete', length = 50)

In [ ]:
# TODO: figure out MemoryError
# Q: does a smaller dtype work?    
# A: nope.
# Q: does a smaller dtype run faster than ~9 minutes?    
# A: yes. 5 minutes
beep(4)

In [ ]:
mesh_current.volume

In [ ]:
df_qstar = pd.DataFrame({
    'tau':tau_values,
    'V':V_lst,
    'rhobar':rhobar_lst
})
df_qstar.shape

In [ ]:
#plot mesh volume versus tau
fontsize=14
fig,ax = plt.subplots(figsize=(6,4))
df_qstar.plot(x='tau',y='V',ax=ax,color='k')#,label=r'$\delta_0$'
# AddLegend(ax=ax,fontsize=fontsize)
format_plot(ax=ax,xlabel=r'$\tau$',ylabel=r'Volume',fontsize=fontsize)
ax.legend().remove()
# plt.yscale('log')
plt.show()

In [ ]:
#mesh volume versus tau
fontsize=14
fig,ax = plt.subplots(figsize=(6,4))
df_qstar.plot(x='tau',y='rhobar',ax=ax,color='k')#,label=r'$\delta_0$'
# AddLegend(ax=ax,fontsize=fontsize)
format_plot(ax=ax,xlabel=r'$\tau$',ylabel=r'Half Mean Curvature',fontsize=fontsize)
# plt.yscale('log')
ax.legend().remove()
plt.show()

In [ ]:
#heretim
#TODO: attempt to solve the isovolemic constraint problem
tau

In [ ]:
tau=1
qstar = np.slerp_vectorized(q0,q1c,tau) #continuous at 0
face_attributes={'rho':rho} #optionally, update local face attributes <<< FVM access point! (useful!) 
# if mesh_current is None:
mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)
# mesh_current.show()

In [ ]:
m=mesh_current.copy()
# mask = m.vertices[:,2] > m.center_mass[2]
mask = (m.vertices[:,2] > m.center_mass[2])|(m.vertices[:,2] <= m.center_mass[2]) #tracked array of True booleans
face_mask = mask[m.faces].all(axis=1)
num_faces_selected = int(face_mask.sum())
num_faces_total    = int(face_mask.shape[0])
print(f"selected {num_faces_selected} out of {num_faces_total} faces ({num_faces_selected/num_faces_total:.1%})")
# # m.update_faces(face_mask)
# m.show()

In [ ]:
#this breaks it...
# #Q: does this update mesh vertices quickly without raising a MemoryError?
# #A: yes. but it appears to shuffle the mesh...
# #inputs: m,qstar,face_mask
# center_result=False
# normalize_result=True
# for step in range(100):
#     pass
# om=quaternion.as_float_array(qstar).copy()
# om = om.reshape((nV,4))
# if center_result:
#     nV=om.shape[0]
#     #center result
#     om = om - np.broadcast_to(np.mean(om,axis=0), shape=(nV,4), subok=False)
# if normalize_result:
#     #normalize output mesh
#     nrm = np.sum(om*om,axis=0)
#     ome_current=om/np.sqrt(np.max(nrm))
# else:
#     ome_current=om

# # m.vertices=ome_current[:,1:]

# # m.vertices-=m.center_mass
# # #normalize output mesh
# # m.vertices*=np.sign(m.volume) #accounts for double cover edge case from quaternionic representation
# # m.vertices/=np.abs(m.bounding_sphere.volume)**0.33 #without bounding sphere

# # m.update_faces(face_mask)
# # m.vertices*=(1./m.bounding_sphere.volume)**(1./3.)
# # m.vertices*=(V0/m.bounding_sphere.volume)**(1./3.)
# #100 updates in 79 ms. no errors other than the obvious one...

# # m.show()

In [ ]:
# m.is_watertight

In [ ]:
#DONT: look for a faster way to compute overall curvature
#NOTE: i can't find an overall measure of curvature that can be quickly accessed...
# trimesh.curvature.coo_matrix(mesh_current)
# # trimesh.curvature.coo_matrix?

In [ ]:
#TODO: show a mesh of q at an intermediate state at time tau
#Q: does it look reasonable at tau=0.5? if not, what about other tau?

In [ ]:
#TODO: graph 1 quaternion's wxyz values on the tau axis using slerp
#TODO: generate ^this from the vector of quaternion representation
# - initial to final quaternions, q0 to q1.
#TODO: generate a video from ^this using ffmpeg and a for loop
#TODO: repeat, generating another video of squad interpolating for qm=[q0 q1],tm=[0 1]


In [ ]:
# who = quaternion.as_float_array(qstar-q1)[0]
# who,np.isclose(who,0.,atol=1e-7)

In [ ]:
# np.isclose(qstar[0]-q1[0],0.)

In [ ]:
# q0 = map_to_quaternion_vector(x0)


In [ ]:
# qstar.shape,qstar.dtype

In [ ]:
face_attributes={'rho':rho}
mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)

In [ ]:
# tau

In [ ]:
# mesh_current.show()

In [ ]:
#print mesh bluf
print(f"{mesh_current.is_watertight=}")
print(f"{mesh_current.volume=}")
print(f"{(mesh_current.volume/mesh_current.bounding_sphere.volume)=} (volume relative to smallest bounding sphere)")
print(f"{mesh_current.center_mass=}")
nrm = np.sum(mesh_current.vertices*mesh_current.vertices,axis=0)
print(f"rms norm of mesh vertices: {list(np.sqrt(nrm))} pixels")
print(f"{mesh_current.is_watertight=} (is the current mesh watertight?)")
print(f"{mesh_current.euler_number=} (what's the euler number for the mesh?)")
print(f"volume relative to smallest bounding sphere: {(mesh_current.volume/mesh_current.bounding_sphere.volume)=}")
# beep(3)

volume_initial = mesh.volume#=4.182994798204928
volume_final = mesh_current.volume
# volume_final = mesh_out.volume
print(f"{volume_initial=:.4f} voxels, {volume_final=:.4f} voxels")
print(f"{mesh.bounding_sphere.volume=:.4f} voxels, {mesh_current.bounding_sphere.volume=:.4f} voxels")
print(f"{volume_initial/mesh.bounding_sphere.volume=:.4f} voxels, {volume_final/mesh_current.bounding_sphere.volume=:.4f} voxels")
#compute the percent change in relative volume
relative_volume_initial = volume_initial/mesh.bounding_sphere.volume
change_in_relative_volume = volume_final/mesh_current.bounding_sphere.volume - volume_initial/mesh.bounding_sphere.volume
relative_change_in_relative_volume = change_in_relative_volume / relative_volume_initial
print(f"percent change in relative volume: {relative_change_in_relative_volume:.4%}")

In [ ]:
# mesh_current.show()

In [ ]:
# Q: can I use slerp directly on x and get a convincing transformation?
# TODO(failing ^that): perform slerp on Lp,ome and repeat the expensive least squares solve.

# dev viewer for quickly saving 1 .png from mesh_current

In [ ]:
# from trimesh.util import BytesIO
import pyglet

In [ ]:
import tempfile
from PIL import Image
#requirements for png generation (i thiiink with OpenGL backend)
# import pyglet
# trimesh.__version__ #'3.9.43'
# pyglet.version #'1.5.11'

In [ ]:
# For $t\in[0,1]$, we may linearly interpolate a conformal map, $L_p$, according to
# $$L^*(t) = tL_p + (1-t)I_p$$
# where $I_p$ is the trivial identity transformation.

In [ ]:
#handle local file system
# mesh_folder = f"{root_dir}/Documents/meshes/res"
folder_vid=os.path.join(mesh_folder,'..')
folder_vid=os.path.join(folder_vid,'vid_000')

if not os.path.exists(folder_vid):
    os.mkdir(folder_vid)
TEXTfoo = lambda N_vertices,N_elements: f"""
TEXT=$'spherical deformation:
      - num. vertices {N_vertices}
      - num. elements {N_elements}
'"""
N_vertices=mesh_out.vertices.shape[0]
N_elements=mesh_out.faces.shape[0]
TEXT = TEXTfoo(N_vertices,N_elements)
print(TEXT)

save_folder_vid=os.path.join(folder_vid,'tmp_000')
if not os.path.exists(save_folder_vid):
    os.mkdir(save_folder_vid)
else:
    shutil.rmtree(save_folder_vid)
    os.mkdir(save_folder_vid)
    
# folnm = os.path.basename(save_folder_vid)
# shutil.rmtree(folnm)
# os.chdir(folnm)
save_folder_vid

In [ ]:
#TODO: test the save function on mesh_out as t=1
#TODO: add wireframe to png
# DONT: average the vertices of the input/output mesh
# mesh_current=0.5*(mesh_out + mesh) #
# mesh_out+mesh,mesh  #<this overlays the meshes, it doesn't average their vertices.
mesh_current = mesh_out.copy()

In [ ]:
assert os.path.exists(save_folder_vid)
# os.path.exists(folnm)
os.chdir(save_folder_vid)

In [ ]:
frameno=-1
save_fn_img = f'img{frameno:09}.png'
# frameno += 1

#generate scene from mesh_current and save scene to .png
camera_angle=(4.2,0.2,3.23) #in radians
scene = mesh_current.scene()
# scene = trimesh.scene.Scene(mesh_current)
scene.set_camera(angles=camera_angle)

In [ ]:
#view the scene
trimesh.viewer.notebook.scene_to_notebook(scene,height=200)

In [ ]:
#TODO: wrap as saving foo
# input: scene
# output: scene saved as .png
# kwargs={}



# input: scene
# output: window
#kwargs are passed to trimesh.viewer.windowed.SceneViewer, which takes <100ms to run.
# kwargs={}

In [ ]:
type(scene)

In [ ]:
#save to .png
save_trimesh_to_png(scene,save_fn_img,format_str='PNG')
print(f"open {os.path.abspath(save_fn_img)}")

In [ ]:
!open /Users/timothytyree/Documents/meshes/vid_000/tmp_000/img-00000001.png

In [ ]:
breakhere

# TODO: generate a minimalist video
- TODO(optional prereq): dev slerp interpolation

In [ ]:
#handle local file system
# mesh_folder = f"{root_dir}/Documents/meshes/res"
folder_vid=os.path.join(mesh_folder,'..')
folder_vid=os.path.join(folder_vid,'vid_000')

if not os.path.exists(folder_vid):
    os.mkdir(folder_vid)
TEXTfoo = lambda N_vertices,N_elements: f"""
    spherical deformation:
      - num. vertices {N_vertices}
      - num. elements {N_elements}
"""
# TEXT=$'spherical deformation:
#       - num. vertices {N_vertices}
#       - num. elements {N_elements}
# '"""
N_vertices=mesh.vertices.shape[0]
N_elements=mesh.faces.shape[0]
TEXT = TEXTfoo(N_vertices,N_elements)
print(TEXT)

save_folder_vid=os.path.join(folder_vid,'tmp_000')
if not os.path.exists(save_folder_vid):
    os.mkdir(save_folder_vid)
else:
    shutil.rmtree(save_folder_vid)
    os.mkdir(save_folder_vid)
    
# folnm = os.path.basename(save_folder_vid)
# shutil.rmtree(folnm)
# os.chdir(folnm)

assert os.path.exists(save_folder_vid)
# os.path.exists(folnm)
os.chdir(save_folder_vid)

In [ ]:
#input: tau_values,q0,q1c,tria,colors,face_attributes={}
#output: .mov that boomerangs over tau_values

In [ ]:
#heretimheretim
#prepare for video
###CAUTION WHEN RUNNING IN PARALLEL
frameno = 1
# time_between_observations = 0.01
time_between_observations = 0.1
time_final = 1.
tme=0.
time_of_next_observation = tme + time_between_observations
# os.chdir(nb_dir)
os.chdir(folder_vid)
if not os.path.exists('mov'):
    os.mkdir('mov')
with open("mov/text.txt", "w") as file:
    file.write(TEXT)
    file.close()
    

In [ ]:
assert os.path.exists(save_folder_vid)
os.chdir(save_folder_vid)
!pwd

In [ ]:
import trimesh.viewer

In [ ]:
def moving_average(a, n=3) :
    ret = np.cumsum(a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
# tau_values = np.arange(0,1+1e-2,1e-1) # 2 seconds per frame. works!
# tau_values = np.arange(0,1+1e-2,1e-2) # higher-res
tau_values = np.arange(0,1+1e-2,2e-2) # faster visualization = fewer frames
tau_values = np.concatenate((tau_values,tau_values[::-1]))# plus boomerang effect yields roughly 4 minutes of runtime
tau_values = moving_average(tau_values, n=15)
plt.plot(tau_values)
plt.show()
tau_values.shape

In [ ]:
#ready... get set... GO!
# vscale=1.05# scaleup vertex coordinates at 5% return per epoch
frameno=0
num_steps=tau_values.shape[0]
step=0
# mesh_current=None
os.chdir('/Users/timothytyree/Documents/meshes/res/../vid_000/tmp_000')
for tau in tau_values:
    #map to vertex representation
    qstar = np.slerp_vectorized(q0,q1c,tau) #continuous at 0
    camera_angle=(4.2,0.2,3.23) #in radians
    face_attributes={'rho':rho} #optionally, update local face attributes <<< FVM access point! (useful!) 
    #TODO: wrap png generator into a shell script
    #inputs: args=qstar, camera_angle, face_attributes
    #output: .png file saved by save_script(*args) if __name__=='_main__':
    #TODO: run ^that script with os.system('save_trimesh_to_png.py') # or 'save_trimesh_to_png.sh')
    mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)
#     if mesh_current is None:
#         mesh_current = map_quaternions_to_trimesh(qstar,tria,colors,face_attributes)
#     else:
#         update_quaternions_to_trimesh_vertices(qstar,mesh_current,center_result=True,
#                                            normalize_result=True,center_result_again=True,
#                                            normalize_by_bounding_sphere=True)#,**kwargs)
    #get image of system
    
    #mesh_current.vertices*=vscale
    scene = mesh_current.scene()
    scene.set_camera(angles=camera_angle)
    #save image of system to .png
    save_fn_img = f'img{frameno:09}.png'
    frameno += 1
    save_trimesh_to_png(scene,save_fn_img,format_str='PNG',flags={'wireframe':True})
    #print progress bar
    if printing:
        step+=1
        if step%update_printbar_every==0:
            printProgressBar(step+1, num_steps, prefix = 'Progress:', suffix = 'Complete', length = 50)

# print(stepsize_mean, stepsize_std, stepsize_median)
beep(3)

In [ ]:
from trimesh.visual import texture, TextureVisuals
from trimesh import Trimesh

In [ ]:
# texture.SimpleMaterial?
# figsize=512,512
# figsize=(2,2)
figsize=(4,4)
# figsize=(8,8)
fig,ax=plt.subplots(figsize=figsize)
ax.set_xlim([0,1])
ax.set_ylim([0,1])
ax.axis('off')
# ax.grid('on')
# for delta in np.arange(0.05,1,0.1):
for delta in np.arange(0.05,1,0.1):
    ax.plot([0,1],[0+delta,0+delta],'k',lw=1)
    ax.plot([0+delta,0+delta],[0,1],'k',lw=1)
plt.tight_layout()
plt.show()

In [ ]:
texture_dir = '/Users/timothytyree/Documents/meshes/vid_000/mov/grid.png'


In [ ]:
mesh_current.visual.defaults['material_shine']=100.0
mesh_current.visual.defaults['material_specular']= np.array([250, 250, 250, 255])
# 'material_diffuse': array([102, 102, 102, 255], dtype=uint8),
#  'material_ambient': array([ 64,  64,  64, 255], dtype=uint8),
#  'material_specular': array([197, 197, 197, 255], dtype=uint8),

material = texture.SimpleMaterial(image=texture_dir)
material.diffuse
material.ambient
material.specular = np.array([250, 250, 250, 255]) #array([102, 102, 102, 255], dtype=uint8)
material.glossiness=255. #default 1.


In [ ]:
# trimesh.creation.uv_sphere?

In [ ]:
# trimesh.visual.texture.util.generate_basis? 
#Generate an arbitrary basis (also known as a coordinate frame)

# trimesh.visual.texture.util.grid_arange(bounds, step)
# Return a grid from an (2,dimension) bounds with samples step distance apart.
# trimesh.visual.texture.util.grid_linspace(bounds, count)

In [ ]:
trimesh

In [ ]:
#NOTE: UV is hard.  just generate a video with a wireframe

In [ ]:
#this doesn't create a uv sphere...
# uv_sphere = trimesh.creation.uv_sphere(radius=1.0, count=[32, 32])
# uv_sphere.visual.

In [ ]:
# trimesh.convex.triangles.

In [ ]:
Rbar = float(np.sqrt(np.mean(np.sum((mesh.vertices-mesh.center_mass)**2,0))))
Rbar

In [ ]:
mesh.visual.

In [ ]:
uv=

In [ ]:
texture = TextureVisuals(uv=uvs, image=img, material=material)
mesh.visual = TextureVisuals(**kwargs)

In [ ]:
#TODO: turn on wireframe
#TODO: generate new frames with half as many frames
# mesh.visual.to_texture() #<< is this the easiest way to get a grid on this surface?


# def get_texture(my_uvs, img):
#     # img is PIL Image
#     uvs = my_uvs
#     material = texture.SimpleMaterial(image=img)    


my_uvs = [....] # 2d array list
vertices = [....] # 3d array list
faces = [....] # indices list
face_normals = [....] # 3d array list
texture_visual = get_texture(my_uvs, img)
mesh = Trimesh(
            vertices=vertices,
            faces=faces,
            face_normals=face_normals,
            visual=texture_visual,
            validate=True,
            process=False
        )

In [ ]:
scene = mesh_current.scene(flags='wireframe')
# scene.toggle_wireframe()
trimesh.viewer.notebook.scene_to_notebook(scene=scene, height=200)

In [ ]:
# mesh_current.show(flags={'wireframe': True})
# mesh_current.show(flags={'wireframe': True, 'axis': True})
# mesh_current.show(flags='wireframe')

In [ ]:
print(f"{frameno=}")
#^this averages 2 seconds per frame
os.mkdir(f"../mov/tmp")

In [ ]:
# scene = mesh.scene()
# #TODO: toggle wireframe
# # searching for "window"
# # (flags='wireframe')

In [ ]:
# scene.metadata
# scene.geometry.

In [ ]:
os.chdir('..')
!pwd

In [ ]:
# os.chdir('/Users/timothytyree/Documents/meshes/vid_000')
os.chdir('mov')
!pwd
!ls

In [ ]:
!echo {save_folder_vid}

In [ ]:
# !cp -r src dst
# os.chdir('/Users/timothytyree/Documents/meshes/res/../vid_000/tmp_000')
os.chdir('/Users/timothytyree/Documents/meshes/res/../vid_000/mov')

In [ ]:
!ls /Users/timothytyree/Documents/meshes/res/../vid_000/mov

In [ ]:
!./concat_folder_png.sh

In [ ]:
#DONE: concatenate_folder
# !./concat_folder_png.sh#todo copy to here
# !./concat_folder_png.sh
!open out.mov


    spherical deformation:
      - num. vertices 4002
      - num. elements 8000


In [ ]:
# !ls ../mov/

In [ ]:
###################################################
# old video generation script using mechanical FVM
###################################################
# time_end_recording = 3#.2
# fps=40
# time_between_observations=1/fps
# vertices=mesh.vertices
# while time_of_next_observation <= time_end_recording:
#     tf = time_of_next_observation
    #integrate forward to the next time of observatiiion
    #     integrate_system_explicit_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities,node_array_mass, element_array_inverse_equilibrium_position)
#     integrate_system_explicit_asynchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities, node_array_mass, element_array_inverse_equilibrium_position, atol_x, atol_v, btol_x, btol_v, learning_rate)
    #     integrate_system_implicit_synchronous(tf, element_array_time, element_array_stepsize, node_array_time,element_array_index, vertices, velocities,node_array_mass, element_array_inverse_equilibrium_position)
    #     integrate_system_implicit_asynchronous(tf, element_array_time, element_array_stepsize, node_array_time,      element_array_index, vertices, velocities, node_array_mass, element_array_inverse_equilibrium_position, atol_x, atol_v, btol_x, btol_v, learning_rate)
    #     integrate_system_dormand_prince_synchronous(tf,element_array_time,element_array_stepsize,node_array_time,element_array_index,vertices,velocities, node_array_mass,element_array_inverse_equilibrium_position,element_array_mass)
    #     integrate_system_dormand_prince_asynchronous(tf,element_array_time,element_array_stepsize,node_array_time,element_array_index,vertices,velocities, node_array_mass,element_array_inverse_equilibrium_position,element_array_mass)

#     #update a copy of all positions to the observation time using the current velocity
#     x = vertices.copy()
#     for a in range(N_vertices):
#         x[a] += velocities[a] * (tf - tau[a])
    
#     #measure observables
#     ##mesh measures
#     net_volume = compute_net_volume(x, element_array_index)
#     net_energy = compute_net_energy(N_elements, element_array_mass, velocities, x, #vertices,
#                                   element_array_index, element_array_inverse_equilibrium_position, mu, lam)
#     ##stepsize measures
#     stepsize_mean = np.mean(element_array_stepsize)
#     stepsize_std = np.std(element_array_stepsize)
#     stepsize_median = np.median(element_array_stepsize)

#     #record observables
#     volume_lst.append(net_volume)
#     energy_lst.append(net_energy)
#     tme = tf
#     tme_lst.append(tme)
#     stepsize_mean_lst.append(stepsize_mean)
#     stepsize_std_lst.append(stepsize_std)
#     stepsize_median_lst.append(stepsize_median)
#     stepsize_count_lst.append(count_array(array = element_array_stepsize,bins=learning_bins))

## feable attempts to get the ` Warning: Expected min height of view:` messages to go away.

In [ ]:
# from ctypes import *

# import pyglet
# from pyglet import gl
# from pyglet.window import BaseWindow, WindowException
# from pyglet.window import MouseCursor, DefaultMouseCursor

# from pyglet.canvas.cocoa import CocoaCanvas

# from pyglet.libs.darwin import cocoapy, CGPoint

# from .systemcursor import SystemCursor
# from .pyglet_delegate import PygletDelegate
# from .pyglet_textview import PygletTextView
# from .pyglet_window import PygletWindow, PygletToolWindow
# from .pyglet_view import PygletView

# NSApplication = cocoapy.ObjCClass('NSApplication')
# NSCursor = cocoapy.ObjCClass('NSCursor')
# NSAutoreleasePool = cocoapy.ObjCClass('NSAutoreleasePool')
# NSColor = cocoapy.ObjCClass('NSColor')
# NSEvent = cocoapy.ObjCClass('NSEvent')
# NSImage = cocoapy.ObjCClass('NSImage')

# quartz = cocoapy.quartz
# cf = cocoapy.cf

In [ ]:
# from pyglet.libs.darwin import cocoapy, CGPoint
# NSCursor = cocoapy.ObjCClass('NSCursor')
# NSColor = cocoapy.ObjCClass('NSColor')
# NSEvent = cocoapy.ObjCClass('NSEvent')
# NSImage = cocoapy.ObjCClass('NSImage')




In [ ]:
######################################################################
# THE PROBLEM WITH THE WARNINGS APPEARS TO BE DEEPER IN MY MACINTOSH
####################s##################################################
#OPTION 1: DO NOTHING
#OPTION 2: SKETCHY FORCE REINSTALL MACVIM
#src: https://stackoverflow.com/questions/64862562/updated-to-mac-os-big-sur-and-getting-warning-expected-min-height-of-view-err

In [ ]:
#needed for rendering
from pyglet.libs.darwin import cocoapy
from pyglet.event import EventDispatcher
NSApplication = cocoapy.ObjCClass('NSApplication')
NSAutoreleasePool = cocoapy.ObjCClass('NSAutoreleasePool')

In [ ]:
def dispatch_events(window):
#     window._allow_dispatch_event = True
#     # Process all pyglet events.
#     window.dispatch_pending_events()
    event = True
    # Dequeue and process all of the pending Cocoa events.
    pool = NSAutoreleasePool.new()
    NSApp = NSApplication.sharedApplication()
    step=0
    while event and window._nswindow and window._context:
        print(f"{step=}");step+=1
        event = NSApp.nextEventMatchingMask_untilDate_inMode_dequeue_(
            cocoapy.NSAnyEventMask, None, cocoapy.NSEventTrackingRunLoopMode, True)
        if event:
            event_type = event.type()
            # Pass on all events.
            NSApp.sendEvent_(event)
#             # And resend key events to special handlers.
#             if event_type == cocoapy.NSKeyDown and not event.isARepeat():
#                 NSApp.sendAction_to_from_(cocoapy.get_selector('pygletKeyDown:'), None, event)
#             elif event_type == cocoapy.NSKeyUp:
#                 NSApp.sendAction_to_from_(cocoapy.get_selector('pygletKeyUp:'), None, event)
#             elif event_type == cocoapy.NSFlagsChanged:
#                 NSApp.sendAction_to_from_(cocoapy.get_selector('pygletFlagsChanged:'), None, event)
            NSApp.updateWindows()
    return pool
#     pool.drain()
#     window._allow_dispatch_event = False

def dispatch_event(window, *args):
    if not window._enable_event_queue or window._allow_dispatch_event:
        if EventDispatcher.dispatch_event(window, *args) != False:
            window._legacy_invalid = True
    else:
        window._event_queue.append(args)

In [ ]:
#TODO: wrap as saving foo
# input: scene
# output: scene saved as .png
#kwargs are passed to trimesh.viewer.windowed.SceneViewer, which takes <100ms to run.
kwargs={}

window = trimesh.viewer.windowed.SceneViewer(scene, resolution = (1280,720), format='PNG',
                                             visible=True,start_loop=False,**kwargs)

In [ ]:
window._allow_dispatch_event = True
# Process all pyglet events.
window.dispatch_pending_events()

In [ ]:
cocoapy.ObjCClass?

In [ ]:
NSApplication = cocoapy.ObjCClass('NSApplication')

In [ ]:
event = True
# Dequeue and process all of the pending Cocoa events.
pool = NSAutoreleasePool.new()
NSApp = NSApplication.sharedApplication()
step=0

In [ ]:
NSApplication.sharedApplication?

In [ ]:

# while event and window._nswindow and window._context:
event , window._nswindow , window._context

In [ ]:
# NSApp.objc_class.

In [ ]:
NSApp.nextEventMatchingMask_untilDate_inMode_dequeue_?
# File:           ~/opt/miniconda3/envs/pyenv/lib/python3.9/site-packages/pyglet/libs/darwin/cocoapy/runtime.py
# This represents an Objective-C method (an IMP) which has been bound
# to some id which will be passed as the first parameter to the method.

In [ ]:
print(f"{step=}");step+=1
event = NSApp.nextEventMatchingMask_untilDate_inMode_dequeue_(
    cocoapy.NSAnyEventMask, None, cocoapy.NSEventTrackingRunLoopMode, True)
#CONFIRMED: NSApp.nextEventMatchingMask_untilDate_inMode_dequeue_ is throwing the annoying warning messages.
event

In [ ]:
#heretimlater
#DONE(when brew update is done): try fixing the "Warning: Expected min height of view"
#POSSIBLE ROOT CAUSE: my macintosh's touchbar doesn't jive.
#POSSIBLE FIX #1: brew reinstall macvim
#POSSIBLE FIX #2: something more violent...

In [ ]:
event.type()

In [ ]:
if event:
    event_type = event.type()
    # Pass on all events.
    NSApp.sendEvent_(event)
    NSApp.updateWindows()


In [ ]:
# print(getsource(window.dispatch_event))
# window._allow_dispatch_event = True
pool = dispatch_events(window)

In [ ]:
pool.drain()
window._allow_dispatch_event = False

In [ ]:
import inspect

print(os.path.abspath(inspect.getfile(window.dispatch_events)))


In [ ]:

# # need to run loop twice to display anything
# for save in [False, False, True]:
#     pyglet.clock.tick()
#     window.switch_to()
#     window.dispatch_events()
#     window.dispatch_event('on_draw')
#     window.flip()
#     if save:
#         # save the color buffer data to memory
#         file_obj = trimesh.viewer.windowed.util.BytesIO()
#         #file_obj = BytesIO()
#         window.save_image(file_obj)
#         file_obj.seek(0)
#         png = file_obj.read()
# window.close()
# rendered = Image.open(trimesh.util.wrap_as_stream(png))
# format_str='PNG'
# #save rendered scene as .png
# rendered.save(save_fn_img, format=format_str)
# del rendered, file_obj, png
# print(f"open {os.path.abspath(save_fn_img)}")

In [ ]:
#TODO: wrap as saving foo
# input: scene
# output: scene saved as .png
#kwargs are passed to trimesh.viewer.windowed.SceneViewer, which takes <100ms to run.
# kwargs={}
# window = trimesh.viewer.windowed.SceneViewer(scene, resolution = (1280,720), format='PNG',
#                                              visible=True,start_loop=False,**kwargs)
# need to run loop twice to display anything
# for save in [True]: #broke it
# for save in [False, True]:#broke it too
for save in [False, False, True]:
    #pyglet.clock.tick() #still works without
#     window.switch_to() #still works without
    window.dispatch_events() #needed #src of warning messages
    window.dispatch_event('on_draw') #needed
#     window.dispatch_event('on_draw') #needed
#     window.flip()#still works without
    if save:
        # save the color buffer data to memory
        file_obj = trimesh.viewer.windowed.util.BytesIO()
        #file_obj = BytesIO()
        window.save_image(file_obj)
        file_obj.seek(0)
        png = file_obj.read()
window.close()
rendered = Image.open(trimesh.util.wrap_as_stream(png))
format_str='PNG'
#save rendered scene as .png
rendered.save(save_fn_img, format=format_str)
del rendered, file_obj, png
# del file_obj, png
print(f"open {os.path.abspath(save_fn_img)}")

In [ ]:
!open /Users/timothytyree/Documents/meshes/vid_000/tmp_000/img-00000001.png
# ERROR: ^this is blank and red uniformly
# Q: is the root cause from scene to scene_viewer or scene_viewer to png?
# A: from scene_viewer to png... scene_viwer renders right in .ipynb.

In [ ]:
#TODO(bonus): annotate scene directly with f"{t=}"
#TODO(failing ^that, bonus): annotate rendered?
type(rendered)

In [ ]:
!rm /Users/timothytyree/Documents/meshes/vid_000/tmp_000/img-00000001.png

In [ ]:
#DONE: attempt to spruce up the rendering using trimesh.viewer.windowed.SceneViewer kwargs
#                                              smooth=False)  #thisa badbad.
#                                              background='black') #TypeError
#                                              caption='testy') #did nothing
#                                              flags=dict(#
#                                                  axis=True#doesn't seem to work...
#                                                  #grid=True#ModuleNotFoundError: No module named 'shapely'
#                                                  #cull=False, unknown effect
#                                                  #wireframe=True # not filled in
#                                                   ))
# # flags (dict) – If passed apply keys to self.view: [‘cull’, ‘wireframe’, etc]
# 'cull': True, 'axis': False, 'grid': False, 'fullscreen': False, 'wireframe': False
#defaults

In [ ]:
# from ..viewer.windowed import render_scene
#         png = render_scene(
#             scene=self, resolution=resolution, **kwargs)

In [ ]:
# # TODO(failling start_loop=False): try modifying kwarg window_conf (None, or gl.Config) – Passed to window init

# # TODO(failling start_loop=False): profile (bool) – If set will run a pyinstrument profile for every call to on_draw and print the output.
# # Q: could this kwarg be useful? record (bool) – If True, will save a list of png bytes to a list located in scene.metadata[‘recording’]
# # Q: can i pass background='black' or 'dark'? not clearly...
# # NOTE: calling scene_viewer.flip() before saving it might magically solve the problem
# scene_viewer = trimesh.viewer.windowed.SceneViewer(scene, resolution = (1280,720), visible=False,  
#                                                    start_loop=False,#,True
#                                                    format='PNG')#,background='w')#,
# # #                           start_loop=False,background='dark'
# #                                                    smooth=True, flags=None, visible=True, resolution=None, 
# #                                                    start_loop=False,#,True 
# #                                                    callback=None, callback_period=None, caption=None, fixed=None, 
# #                                                 offset_lines=True, line_settings=None, background=None, window_conf=None, 
# #                                                    profile=False, record=False)#, **kwargs)

# scene_viewer.flip()

In [ ]:
# from inspect import getsource
# # print(getsource(scene.save_image))
# print(getsource(trimesh.viewer.windowed.render_scene))

In [ ]:
#heretimheretim
#TODO: get this working and wrap it to foo
# save_scene_to_png_trimesh(save_fn_img, scene, 
#                           resolution = (1280,720),visible=False, format='PNG',
#                           start_loop=False,background='dark')#,**kwargs)

## scratchwerk

In [ ]:
# png = scene.save_image(resolution=resolution, visible=visible,start_loop=start_loop)#,**kwargs)

In [ ]:
# #save scene as .png
# rendered = Image.open(
#     trimesh.util.wrap_as_stream(png))
# rendered.save(save_fn_img, format=format_str)
# del rendered

In [ ]:
# # window_conf
# gl.Config

In [ ]:
# scene_viewer.background='black'

In [ ]:
# scene_viewer.background

In [ ]:
# trimesh.viewer.notebook.scene_to_notebook(scene_viewer.scene,height=200)
# trimesh.viewer.notebook.scene_to_notebook(scene_viewer.scene,height=480)
# trimesh.viewer.notebook.scene_to_notebook(scene_viewer.scene)

In [ ]:
# trimesh.viewer.render_scene?
# ^that's slow...

In [ ]:
# # with tempfile.NamedTemporaryFile(suffix='.png') as file_obj:
# with tempfile.TemporaryFile() as file_obj:
#     file_obj = scene_viewer.save_image(file_obj)
#     file_obj.seek(0)
#     data = file_obj.read()
#     rendered = Image.open(trimesh.util.wrap_as_stream(data))
#     del data

# format_str='PNG'
# #save rendered scene as .png
# rendered.save(save_fn_img, format=format_str)
# del rendered
# print(f"open {os.path.abspath(save_fn_img)}")

# !open /Users/timothytyree/Documents/meshes/vid_000/tmp_000/img-00000001.png
# # ERROR: ^this is blank and red uniformly
# # Q: is the root cause from scene to scene_viewer or scene_viewer to png?
# # A: from scene_viewer to png... scene_viwer renders right in .ipynb.

In [ ]:
# # trimesh.viewer.notebook.scene_to_notebook(scene=scene, height=480)#,camera=camera,lights=None,camera_transform=None)#720)#1080)
#don't use this. it's slow.
# png = trimesh.viewer.render_scene(scene, resolution=(1280,720), visible=False)#, **kwargs)
# # start_loop=False,#,True
# #                                                    format='PNG')
# type(png)

In [ ]:
# scene_viewer.visible

In [ ]:
# scene.show()

In [ ]:
# # it appears to work!
# # png = 
# scene_viewer.save_image()
# start_loop=False) #stop launching an ipykernel

In [ ]:
# print(f"open {os.path.abspath(save_fn_img)}")

In [ ]:
# scene.save_image(start_loop=False) #stop launching an ipykernel? no.

In [ ]:
# #Q: is save_image launching ipykernel?
# A: yes.
# scene.save_image(start_loop=False) #stop launching an ipykernel

In [ ]:
# #input: scene
# #output: saved png without opening a separate ipykernel window
# with tempfile.TemporaryFile() as file_obj:
#     print(file_obj.name)

In [ ]:
# png = scene.render_scene()

# #TODO: pass flags kwarg for wireframe
# # flags (dict) – If passed apply keys to self.view: [‘cull’, ‘wireframe’, etc]
# type(png)

In [ ]:
# scene_viewer

In [ ]:
# png = trimesh.viewer.windowed.render_scene(scene=scene)
# type(png)

In [ ]:
# scene.save_image?

In [ ]:
# #generate png from mesh_current
# resolution = (1280,720)
# camera_angle=(-3*45,0,0)
# # scene = trimesh.scene.Scene(mesh_current)
# scene = mesh_current.scene()
# scene.set_camera(angles=camera_angle)
# # #save scene to .png
# # save_scene_to_png_trimesh(save_fn_img, scene, 
# #                           resolution = (1280,720),visible=True, format='PNG')#,**kwargs)
# print(f"open {os.path.abspath(save_fn_img)}")
# # scene.show()

In [ ]:
# !open /Users/timothytyree/Documents/meshes/vid_000/tmp_000/img-00000001.png


In [ ]:
# mesh = trimesh.primitives.Sphere()
# scene = mesh.scene()

# data = scene.save_image()
# # if this doesn't work, try it with a visible window:
# # data = scene.save_image(visible=True)


# from PIL import Image
# rendered = Image.open(trimesh.util.wrap_as_stream(png))
# # rendered.show()

In [ ]:
# scene.save_image?

In [ ]:
#heretim
#TODO: dev identity map in Lp representation, Ip
#TODO: see if Ip=np.eye(x.shape[0]) works
#TODO(failing ^that): figure out what a constant (or zero) rho setting does.  record this as Ip if apparent identity.

In [ ]:
# from inspect import getsource
# print(getsource(trimesh.viewer.windowed.SceneViewer))


In [ ]:
# manager = pyglet.image.get_buffer_manager()
# colorbuffer = manager.get_color_buffer()
# # if passed a string save by name
# if hasattr(file_obj, 'write'):
#     colorbuffer.save(file=file_obj)
# else:
#     colorbuffer.save(filename=file_obj)
# # return file_obj

In [ ]:
# import tempfile
# # mesh = trimesh.primitives.Sphere()
# scene = mesh.scene()
# with tempfile.TemporaryFile() as file_obj:
#     scene.save_image(file_obj, visible=True)#, resolution=(480,480))
#     file_obj.seek(0)
#     data = file_obj.read()
# # scene.show()
# from PIL import Image
# rendered = Image.open(trimesh.util.wrap_as_stream(data))
# # rendered.show()

In [ ]:
# # plt.imshow(png)
# # bytes(png)
# from PIL import Image
# import io
# img_byte_arr = io.BytesIO()
# # png.save(img_byte_arr, format='PNG')
# scene.save_image(img_byte_arr, format='PNG')
# img_byte_array = img_byte_arr.getvalue()
# type(img_byte_array)

In [ ]:
# # scene.save_image?
# img = Image.Image(png)
# Img = Image.fromarray(png)
# # Img.save(save_fn_img)
# # del Img

# # type(img)

In [ ]:
# # IFrame
# points=mesh_current.vertices
# # scene.camera_transform = scene.camera.look_at(2*points)
# scene.set_camera(angles=(-90-45,0,45))
# # trimesh.viewer.notebook.scene_to_notebook(scene=scene, height=480)#,camera=camera,lights=None,camera_transform=None)#720)#1080)
# png = scene.save_image(resolution=None)#, **kwargs)
# # Passed to SceneViewer constructor

# DONE: fork from previous video generation script

In [ ]:
# TODO: fork old video creation method

#heretim
#TODO: find input fn

# # mesh_folder = f"{root_dir}/Documents/meshes/res"
# folder_vid=os.path.join(mesh_folder,'..')
# folder_vid=os.path.join(folder_vid,'vid_000')


In [ ]:
# from PIL import Image
# import trimesh, tetgen, pyvista as pv
# from IPython.utils import io

In [ ]:
# #visualize the mesh surface
# 	pv.set_plot_theme('document')

# 	#get the vtk object (wrapped by pyvista from withing tetgen.  Faces recorded by trimesh))
# 	if faces is None:
# 		if input_file_name is None:
# 			Exception('either faces or input_file_name must be specified')
# 		mesh_trimesh = trimesh.load(input_file_name)
# 		faces = mesh_trimesh.faces

# 	tet = tetgen.TetGen(vertices, faces)
# 	# #fault tolerant tetrahedralization
# 	vertices_tet, elements_tet = tet.tetrahedralize(order=1, mindihedral=0., minratio=10., nobisect=False, steinerleft=100000)#
# 	tet.make_manifold()
# 	grid = tet.grid

# 	# advanced plotting
# 	plotter = pv.Plotter()
# 	if darkmode:
# 		plotter.set_background(background_color)
# 		plotter.add_mesh(grid, 'lightgrey', lighting=True)
# 		#looks like tron plotter.add_mesh(grid, 'r', 'wireframe')
# 	else:
# 		plotter.add_mesh(grid, 'lightgrey', lighting=True)
# 		font_color = 'k'
# 	if text is not None:	
# 		plotter.add_text(
# 		    text,
# 		    position='upper_left',
# 		    font_size=24,
# 		    color=font_color,
# 		    font='times')
# 	#font options
# 	# FONT_KEYS = {'arial': vtk.VTK_ARIAL,
# 	#              'courier': vtk.VTK_COURIER,
# 	#              'times': vtk.VTK_TIMES}

# 	#cpos is (camera position, focal point, and view up)
# 	#for movies, just set the camera position to some constant value.

# 	_cpos, img = plotter.show(title=None, return_img=True, cpos=cpos, window_size=window_size, use_ipyvtk = False, interactive=False, auto_close=True);
# 	plotter.deep_clean()
# 	del plotter
# 	pv.close_all()
# 	return img

In [ ]:
# darkmode = True; background_color = 'k'; text=None;
# window_size = [1280,720]; font_color = 'w'
cpos = [(3.77, 3.77, 3.77),(0.0069, -0.0045, 0.0),(0.0, 0.0, 1.0)]

In [ ]:
faces = mesh_out.faces

In [ ]:
#This method is very slow! another is needed!
img = get_img_of_system(vertices, faces=faces, 
                        input_file_name=None, darkmode = False, text=f'time={tf:.2f}',cpos=cpos)


# GOAL: dev viewer for a given mesh

In [ ]:
scene = trimesh.scene.Scene(mesh)

In [ ]:
# mesh_out.show()
# /Users/timothytyree/opt/miniconda3/envs/pyenv/lib/python3.9/site-packages/IPython/core/display.py:724: UserWarning: Consider using IPython.display.IFrame instead
#   warnings.warn("Consider using IPython.display.IFrame instead")
# camera : Camera or None
#   A passed camera to use
# lights : [trimesh.scene.lighting.Light] or None
#   A passed lights to use
# camera_transform : (4, 4) float or None
#   Camera transform in the base frame
points=mesh_out.vertices
# scene.camera_transform = scene.camera.look_at(2*points)
scene.set_camera(angles=(-90-45,0,45))
trimesh.viewer.notebook.scene_to_notebook(scene=scene, height=480)#,camera=camera,lights=None,camera_transform=None)#720)#1080)


In [ ]:
#optionally play with action


In [ ]:
#optionally play with camera
scene.camera = trimesh.scene.cameras.Camera(name='camera_instance', resolution=(100,480), focal=(1,1), fov=None, z_near=0.01, z_far=1000.0)
scene.camera

In [ ]:
#optionally play with lights
scene.lights

In [ ]:
scene.set_camera(angles=(-90-45,0,45))
# trimesh.viewer.notebook.scene_to_notebook(scene=scene, height=480,lights=None)#720)#1080)

In [ ]:
# compose scene
scene = pyrender.Scene(ambient_light=[.1, .1, .3], bg_color=[0, 0, 0])
camera = pyrender.PerspectiveCamera( yfov=np.pi / 3.0)
light = pyrender.DirectionalLight(color=[1,1,1], intensity=2e3)

scene.add(mesh, pose=  np.eye(4))
scene.add(light, pose=  np.eye(4))

c = 2**-0.5
scene.add(camera, pose=[[ 1,  0,  0,  0],
                        [ 0,  c, -c, -2],
                        [ 0,  c,  c,  2],
                        [ 0,  0,  0,  1]])

# render scene
r = pyrender.OffscreenRenderer(512, 512)
rcolor, _ = r.render(scene)

plt.figure(figsize=(8,8)), plt.imshow(rcolor);

In [ ]:
# trimesh.viewer.render_scene(*args, **kwargs)
# trimesh.viewer.notebook.in_notebook()

In [ ]:
#heretim
#GOAL: dev viewer
#TODO: dev .png generator
# Lp.shape,x.shape
# HINT: use the IPython.Frame thingy
#TDOO: generate a folder of .png files
#TODO: dev ffmpg wrapper script, as before

# scratchwerk

## scratchwerk

In [ ]:
# #>10 minute run time...
# Linv = np.linalg.pinv(L)
#DONE: try the pseudo inverse L instead

In [ ]:
#test the inverted matrix has only valid entries
assert not np.isnan(Linv).any()
assert not np.isinf(Linv).any()

In [ ]:
#run time >10 minutes
# # print(np.around((Linv[-9:,-9:]),3))
# # print(np.around((Linv[:9,:9]),1))
# U, S, V = np.linalg.svd(L, full_matrices=False)
# U.shape, S.shape, V.shape 

In [ ]:
# vmax=0.01
# vmax=0.001
vmax=0.0001
figsize=(4,4)
# figsize=(7,7)
fig,ax=plt.subplots(figsize=(7,7))
ax.imshow(Lp,
    cmap='bwr',
# #     norm=None,
# #     aspect=None,
# #     interpolation=None,
# #     alpha=None,
    vmin=-vmax,
    vmax=vmax)
# format_plot
# format_plot(ax=ax,xlabel=r'n (1/cm$^2$)',ylabel=r'w (Hz/cm$^2$)',fontsize=fontsize)#,use_loglog=True)
ax.set_xticklabels([])
ax.set_yticklabels([])
ax.tick_params(top=True, right=True,direction='in',which='both')
plt.tight_layout()
plt.show()

In [ ]:
print(np.min(L),np.max(L))
print(np.min(Linv),np.max(Linv))
# -0.7261430355213236 3.630588505800929
# -42089716143649.945 0.0

# from here in python:
# print(np.min(L),np.max(L))
# -0.7261430355213236 3.630588505800929
#from matlab:
# min(L): -7.261430e-01, max(L): 3.630589e+00

In [ ]:
#DONE: visualize the pointcloud of ome_raw@Linv
#DONE: visualize the pointcloud of Linv@ome_raw
#NOTE: none of them look right

In [ ]:
# # #test the inverse matrix works with right multiplication
# roo = np.abs( (L@Linv) - np.eye(L.shape[0]))
# print(roo[:3,:3])
# np.min(roo),np.max(roo) #1?? sig figs of agreement (17% error)
# # # [[0.00097656 0.         0.        ]
# # #  [0.         0.0078125  0.        ]
# # #  [0.         0.         0.0078125 ]]
# # # (0.0, 0.171875)
# # [[0.00585938 0.         0.        ]
# #  [0.         0.046875   0.        ]
# #  [0.         0.         0.046875  ]]

In [ ]:
# # #test the inverse matrix works with right multiplication
# roo = np.abs( (Linv@L) - np.eye(L.shape[0]))
# print(roo[:3,:3])
# np.min(roo),np.max(roo) #14?? sig figs of agreement (17% error)
# # # [[0.01855469 0.         0.        ]
# # #  [0.         0.08984375 0.        ]
# # #  [0.         0.         0.08984375]]
# # # (0.0, 0.2578125)
# # [[0.02148438 0.         0.        ]
# #  [0.         0.1015625  0.        ]
# #  [0.         0.         0.1015625 ]]

In [ ]:
#OBSERVATION: precomputing Linv doesn't work perfectly
#~1 minute run time and off by ~ 6 %...
# L@Linv
#~1 minute run time and off by ~ 10 %...
# Linv@L

In [ ]:
# #ROOT CAUSE DETECTED? constant shift from 1 indexing...
# #SOLVED
# # - in matlab the first element is 1,2,3
# # - in python the first element is 0,-1,-2,
# # (...)>> try shifting ome_raw before reshaping it
# # ome_raw==ome_raw.T
# np.mean(ome_raw.reshape((nV,4)),axis=0)
# #Q: which axis has the smallest mean value? shift so that goes to 0 some way or another.

In [ ]:
# #DONE: checked swap isn't incorrect...
# k1=0;k2=1;k3=2
# print(f"{k1=}, {k2=}, {k3=}")
# k3=k1; k1=k2; k2=k3; #% swap
# print(f"{k1=}, {k2=}, {k3=}")


In [ ]:
#DONE: check that I'm not using x/L (pointwise division) anywhere...

In [ ]:
# np.min(ome),np.max(ome),np.mean(ome),np.var(ome)

In [ ]:
# # Potential ROOT CAUSE: 
# # Q: Am I casting from quaternion to cartesian incorrectly?
# #NOTE: om is in the basis of 1ijk
# #NOTE: ome is in the basis of 1ijk
# #NOTE: L is in the basis of quaternions
# # om = om.reshape((nV,4))
# ome_l.shape,ome_r.shape,Linv.shape,L.shape,om.shape

# #maxabs percent difference of 42% between left and right division suggests which one i use really matters...
# #maxabs percent difference of 13% between left and right division suggests which one i use really matters...
# np.max(np.abs(ome_r-ome_l))/np.max(np.abs(ome_r))

In [ ]:
#>10 minutes of run time
# # divisor=np.roll(Linv,(12,12))
# divisor=np.roll(L,(12,12))
# om = np.linalg.lstsq(divisor.T, ome.T)[0].T
# om.shape

In [ ]:
#center result
ome = ome_raw.reshape((nV,4))
# ome = ome_raw.T.reshape((nV,4)) #<<<same thing
# Q: which one of ^these is correct?
# ome = ome_raw.reshape((4,nV))
ome = ome - np.broadcast_to(np.mean(ome,axis=0), shape=(nV,4), subok=False)
# ome.reshape((4,nV))
# ome=reshape(ome,[4 nV]);
# ome=ome-repmat(mean(ome,2),[1 nV]);
ome = ome.flatten()
# ome=reshape(ome,[4*nV 1]);
# om = ome@Linv
om = Linv@ome
# om = Linv.T@ome
print(om.shape)
# np.divide?
om1=om.copy()

In [ ]:
#compute the vector `x` that approximately solves the equation
# ``a @ x = b``.

#this will take 17-22 minutes to rerun...
# om = Linv@ome #the incumbant
om2 = np.linalg.lstsq(Lp, ome)[0]    # not under-determined

In [ ]:
#plot the computed groundstate
plt.plot(om2[::4])
plt.plot(om2[1::4])
plt.plot(om2[2::4])
plt.plot(om2[3::4])

In [ ]:
# cp.sparse.cupyx.optimizing.optimize?
# cholesky decomposition doesn't explicitely solve least-squares. 
# cp.sparse.cupyx.linalg.sparse.lschol(A,b)
# cp.sparse.cupyx.linalg.invh?
# cp.sparse.cupyx.linalg.sparse.lschol?

In [ ]:
# a=Lp.copy()
# b=ome.copy()
# x0=Linv@b

In [ ]:

#DONE: try scipy.sparse or something lstsq in scipy
#DONT: block-solve upper-triangular / lu decomposition or something to make it go faster

In [ ]:
#GOAL: dev map Linv,ome,Lp to ome_refined=x
# <--->  x minimizes the Euclidean 2-norm `|| b - a x ||^2`
# def lsq_solve_pytorch(a,x0,b):
#     pass
# <--->  x minimizes the Euclidean 2-norm `|| b - a x ||^2`
# def lsq_solve_scipy(a,x0,b):
#     pass
#TODO: implement gradient-descent approach to map Linv,ome,Lp to Linv_refined in numpy/cupy?
#TODO: implement gradient-descent approach to map Linv,ome,Lp to Linv_refined in pytorch
#TODO: try using the iterative method from before to find the groundstate eigensolution lam, to E*lam = res*lam

In [ ]:
# np.linalg.norm(x)

In [ ]:
print(f"Root-Mean-Square-Error between numpy and scipy least-square solutions: {rmse=:.4f}")
rmse = np.sqrt(np.linalg.norm(x-om2))
print(f"Solution Norms: (scipy soln, numpy soln)")
print(f"- {np.linalg.norm(x)=:.6f}, {np.linalg.norm(om2)=:.6f}")
print(f"- {np.linalg.norm(x**2)=:.10f}, {np.linalg.norm(om2**2)=:.10f}")
# mpe=np.mean(np.abs(x-om2)/x)
# print(f"{mpe=:}")

In [ ]:

# cp.sparse.cupyx.lapack.gesv?

In [ ]:
# Lpc=cp.array(Lp)
# Lpc.shape

In [ ]:
# omec=cp.array(ome)

In [ ]:
# #speed-up this step in cp
# retval_c = cp.linalg.lstsq(Lpc, omec)    # not under-determined
# (om2c, residuals, rank, s) = retval_c
# #  OutOfMemoryError: Out of memory allocating 2,067,344,896 bytes (allocated so far: 6,150,403,072 bytes). 

In [ ]:
# om2=om2c.get()

In [ ]:

# # om2,residuals2,rank2,singular_value_array2=retval_
# # om2,residuals2,rank2=retval_
# # om2,residuals2=retval_
# om2=retval_
# om2.shape

In [ ]:
# beep(7)

In [ ]:
# # om = ome@Linv #the transpose. not the same as solving ``a @ x = b``.
# om3,residuals3,rank3,singular_value_array3 = np.linalg.lstsq(Lp.T, ome)[0]    # not under-determined
# om3.shape

## DONE: Plot V_out / more scratchwerk

In [ ]:
V_out.dtype

In [ ]:
#visualize the output mesh
# initialize mesh colored by rho
mesh_out = trimesh.Trimesh(vertices=V_out,faces=tria,
                       face_colors=colors,
                       face_attributes={'rho':rho})

print(f"{mesh_out.is_watertight=}")
# mesh_out.show()
# ****
#print mesh bluf
print(f"{mesh_out.volume=}")
print(f"{mesh_out.center_mass=}")
print(f"{mesh_out.is_watertight=} (is the current mesh watertight?)")
print(f"{mesh_out.euler_number=} (what's the euler number for the mesh?)")

In [ ]:
beep(3)
#Q: does adding/subtracting 1 from every vertex index in tria help?
# A: no.
# HINT: 
# tria_=tria-1
# tria_=tria+1


In [ ]:
# #caution really slow...
# #Q: does slerp use something like exponential interpolation?
# dt=1e-2
# dlp=scipy.linalg.logm(Lp)
# Lp_next = scipy.linalg.expm(dt*dlp)
# rmsd = np.sqrt(((Lp_next-Lp)[:,0]**2).mean())
# print(f"exponentially rotating the deformation matrix by {dt=}: {rmsd=}")

In [ ]:
# # Q: can I get a given state of eigenspectrum this way?
# #get the first two eigenvectors
# Ab = decomp_banded_matrix(A=E)
# w, v = scipy.linalg.eig_banded(Ab, lower=True, select='v', select_range=[0,1])

# #>3 minute runtime...
# #get all eigenvectors
# # w, v = eig_banded(E, lower=True)

In [ ]:
# import numpy as np
# from scipy.linalg import eig_banded
# A = np.array([[1, 5, 2, 0], [5, 2, 5, 2], [2, 5, 3, 5], [0, 2, 5, 4]])
# Ab = np.array([[1, 2, 3, 4], [5, 5, 5, 0], [2, 2, 0, 0]])
# w, v = eig_banded(Ab, lower=True)
# np.allclose(A @ v - v @ np.diag(w), np.zeros((4, 4)))
# True
# w = eig_banded(Ab, lower=True, eigvals_only=True)
# w

# w, v = eig_banded(Ab, lower=True, select='v', select_range=[-3, 4])
# w.shape,v.shape,Ab.shape



In [ ]:
# import numpy as np
# from scipy.linalg import cholesky_banded, cho_solve_banded
# Ab = np.array([[0, 0, 1j, 2, 3j], [0, -1, -2, 3, 4], [9, 8, 7, 6, 9]])
# A = np.diag(Ab[0,2:], k=2) + np.diag(Ab[1,1:], k=1)
# A = A + A.conj().T + np.diag(Ab[2, :])
# c = cholesky_banded(Ab)
# x = cho_solve_banded((c, False), np.ones(5))

In [ ]:
#Eco_ = cho_solve_banded((cholesky_banded(E), False), lower=True)

In [ ]:
#Eco_.shape

In [ ]:
#Eco___

In [ ]:
# # i=0
# i=-1
# # ome = vh[int(i)].reshape((nV,4))
# # ome = vh[int(i)].reshape((nV,4))
# ome = ome_raw.reshape((nV,4))

# #center result
# ome = ome - np.broadcast_to(np.mean(ome,axis=0), shape=(nV,4), subok=False)
# ome = ome.flatten()
# #map result
# # om = ome.copy()
# # om = ome@Linv
# om = Linv@ome#@Linv
# # om = Linv@ome
# # om = Linv.T@ome
# #center result
# om = om.reshape((nV,4))
# om = om - np.broadcast_to(np.mean(om,axis=0), shape=(nV,4), subok=False)
# #normalize output mesh
# nrm = np.sum(om*om,axis=0)
# ome_out=om/np.sqrt(np.max(nrm))
# # ome_out=om
# V_out = ome_out[:,1:]
# scalar_out = ome_out[:,0]
# # V_out=ome(2:end,:);
# print(f"\nQ: is the smallest value in the first index?\m")
# print(f"*** Norm: {list(nrm)} ***")
# print(np.min(scalar_out),np.max(scalar_out))
# assert vert.shape==V_out.shape
# assert not np.isnan(V_out).any()
# print(np.max(np.abs(V_out)),np.mean(np.abs(V_out)))
# print(f"{V_out.dtype=}")

# #Q: does permuting triangles by +/- 1-3 fix make it smooth?
# #A: no. however, it does shift the colors while not affecting the geometry...

#shift the colors
# tria_=tria.copy()
# # tria_=tria.copy()
# tria_=np.roll(tria,-1,axis=1)  
 

# # tria_=np.roll(tria,1,axis=0)  #<< makes funny color streaks
# # tria_=np.roll(tria,100,axis=0) #<< roughly shuffled colors
# # V_out = V_out[:,(0,1,2)].copy() #
# #  V_out = V_out[:,(0,2,1)].copy() #*
# # V_out = V_out[:,(1,0,2)].copy() #*
# # V_out = V_out[:,(1,2,0)].copy() #*
# # V_out = V_out[:,(2,1,0)].copy()
# # V_out = V_out[:,(2,0,1)].copy()
# #visualize the output mesh
# # initialize mesh colored by rho
# # mesh_out = trimesh.Trimesh(vertices=V_out,faces=tria,
# mesh_out = trimesh.Trimesh(vertices=V_out,faces=tria_,
#                        face_colors=colors,
#                        face_attributes={'rho':rho},alpha=0.5)

# print(f"{mesh_out.is_watertight=}")
# mesh_out.show()
# # ****
# # is the current mesh watertight?

In [ ]:
#<12 minute runtime
wh, vh = np.linalg.eigh(Lp)
wh.shape,vh.shape

In [ ]:
# print(np.isclose(wh,1,atol=5e-3).sum(),wh.shape[0])
print(f'{np.isclose(wh,1,atol=5e-3).sum():d} out of {wh.shape[0]:d} linearly independent dimensions has an eigenvalue very close to unity...')

In [ ]:
# lamh = vh[np.argmin(wh)]
lamh = vh[wh==1]
lamh.shape

In [ ]:
for i in np.argwhere(np.isclose(wh,1,atol=5e-3)):
    print(f"{i=}")

In [ ]:
# # i=0
# i=-1
# # ome = vh[int(i)].reshape((nV,4))
# ome = vh[int(i)].reshape((nV,4))
# ome = ome_raw.reshape((nV,4))

# #center result
# ome = ome - np.broadcast_to(np.mean(ome,axis=0), shape=(nV,4), subok=False)
# ome = ome.flatten()
# #map result
# # om = ome.copy()
# # om = ome@Linv
# om = Linv@ome#@Linv
# # om = Linv@ome
# # om = Linv.T@ome
# #center result
# om = om.reshape((nV,4))
# om = om - np.broadcast_to(np.mean(om,axis=0), shape=(nV,4), subok=False)
# #normalize output mesh
# nrm = np.sum(om*om,axis=0)
# ome_out=om/np.sqrt(np.max(nrm))
# # ome_out=om
# V_out = ome_out[:,1:]
# scalar_out = ome_out[:,0]
# # V_out=ome(2:end,:);
# print(f"\nQ: is the smallest value in the first index?\m")
# print(f"*** Norm: {list(nrm)} ***")
# print(np.min(scalar_out),np.max(scalar_out))
# assert vert.shape==V_out.shape
# assert not np.isnan(V_out).any()
# print(np.max(np.abs(V_out)),np.mean(np.abs(V_out)))
# print(f"{V_out.dtype=}")

# #Q: does permuting triangles by +/- 1-3 fix make it smooth?
# #A: no. however, it does shift the colors while not affecting the geometry...
# tria_=tria.copy()
# # tria_=np.roll(tria,1,axis=0)  #<< makes funny color streaks
# # tria_=np.roll(tria,100,axis=0) #<< roughly shuffled colors
# # V_out = V_out[:,(0,1,2)].copy() #
# #  V_out = V_out[:,(0,2,1)].copy() #*
# # V_out = V_out[:,(1,0,2)].copy() #*
# # V_out = V_out[:,(1,2,0)].copy() #*
# # V_out = V_out[:,(2,1,0)].copy()
# # V_out = V_out[:,(2,0,1)].copy()
# #visualize the output mesh
# # initialize mesh colored by rho
# # mesh_out = trimesh.Trimesh(vertices=V_out,faces=tria,
# mesh_out = trimesh.Trimesh(vertices=V_out,faces=tria_,
#                        face_colors=colors,
#                        face_attributes={'rho':rho},alpha=0.5)

# print(f"{mesh_out.is_watertight=}")
# mesh_out.show()
# # ****
# # is the current mesh watertight?

In [ ]:
# np.outer(Lp@vh[i],vh[i])#.shape,(Lp@vh[i]).shape

In [ ]:
# np.dot(Lp,vh[i]).sum(),wh[i],np.linalg.norm(vh[i])

In [ ]:
# wh[i]

In [ ]:
# #plot the computed groundstate
# plt.plot(lam[::4])
# plt.plot(lam[1::4])
# plt.plot(lam[2::4])
# plt.plot(lam[3::4])

#plot the eigensolution for the groundstate
plt.plot(lamh[::4])
plt.plot(lamh[1::4])
plt.plot(lamh[2::4])
plt.plot(lamh[3::4])

In [ ]:
plt.plot(wh)

In [ ]:
#intentionally cause an error to avoice noming gpu memory
Lpcinv

In [ ]:
#visualize ^this eigensolution using trimesh, as before
#TODO: wrap this to foo

In [ ]:
Lpc=cp.array(Lp)
Lpc.shape

In [ ]:
#TODO: compute w,v using cupy
wg,vg = cp.linalg.eigvalsh(Lpc)
wg.shape

In [ ]:
# Lpcinv = cupyx.linalg.invh(Lpc)

In [ ]:
# import cupy
mempool = cupy.get_default_memory_pool()
pinned_mempool = cupy.get_default_pinned_memory_pool()

In [ ]:
print(mempool.used_bytes())              
print(mempool.total_bytes())             
print(pinned_mempool.n_free_blocks())

In [ ]:
#run gpu cleanup
# del Lpc,wg,vg#Lpcinv
mempool.free_all_blocks()

print(mempool.used_bytes())              
print(mempool.total_bytes())             
print(pinned_mempool.n_free_blocks())

In [ ]:
#TODO: compute w,v using cupy
# HINT: use solve
#solve ax=b
# cupy.linalg.solve?
#note: this took <11 seconds to run
x_soln = cp.linalg.solve(a=Lpc, b=cp.zeros(Lp.shape[0]))
x_soln.shape

In [ ]:
# lamg = x_soln.get()
# type(lamg)
# # #plot the computed groundstate
# # plt.plot(lam[::4])
# # plt.plot(lam[1::4])
# # plt.plot(lam[2::4])
# # plt.plot(lam[3::4])

# # #plot the eigensolution for the groundstate
# # plt.plot(lamh[::4])
# # plt.plot(lamh[1::4])
# # plt.plot(lamh[2::4])
# # plt.plot(lamh[3::4])

# #plot the eigensolution for the groundstate
# plt.plot(lamg[::4])
# plt.plot(lamg[1::4])
# plt.plot(lamg[2::4])
# plt.plot(lamg[3::4])

In [ ]:
lamg

In [ ]:
# import cupyx

In [ ]:
# cupyx.scipy.sparse.linalg.lsqr

In [ ]:
# cupyx.scipy.sparse.csr_matrix
# cupyx

In [ ]:
# Q: why is my mesh so noisier than the demo solution from matlab?

In [ ]:
# #warning: #>16 minutes run time
# #53-37 = >16 
# w, v = np.linalg.eig(E)
# w.shape,v.shape,np.min(w),np.max(w) 
# #for simplest unit sphere example:
# # ((264,),
# #  (264, 264),
# #  (-6.216318184123621-3.508736056987371j),
# #  (10.372886477259154+0j))

# #warning >7 minutes run time
# wh, vh = np.linalg.eigh(E)
# wh.shape,vh.shape,np.min(wh),np.max(wh)

In [ ]:
#TODO: test that changing E's dtype to float 64 doesn't make everything immediately better
#TODO: improve this method
#Option 1: keep looking for mistakes somewhere...
#Option 2: improve estimation of lam. get a better solution bc >10% error in ground state eigenvector would explain the numerical noise...
#   - 2.A: try canned response from cupy.linalg
#   - 2.A: try simplest possible pytorch routine
#   - 2.C: try pycuda


In [ ]:
# import cupy as cp

In [ ]:
# E.dtype

In [ ]:
#TODO: get the demo working to machine precision
#HINT: check the matlab code to see if i'm missing any [plc] reindexing...
#TODO: attempt to solve the linear system of equations more explicitely       
#HINT: try adding res to rho
#TODO: set rho to a constant value everywhere and see if i get a sphere coming out
#TODO: can i get dirac spheres at the harmonics (n>1)

In [ ]:
#Q: did shifting the other direction help?
#A: no.
#Q: does permuting tria help?
#A: unlikely to be watertight...
# #Q: does settingrho=0 return the identity morphism?
# #A: no.
# #DONE: figure out why I'm not able to get the identity morphism. 
# A: simply use the discrete half mean curvature density of the original mesh!

In [ ]:
# plt.plot(lam) #<<< lam appears to agree with matlab

In [ ]:
# #heretim
# #TODO: visualize the pointcloud of ome_raw
# x_values = ome_raw[1::4]
# y_values = ome_raw[2::4]
# z_values = ome_raw[3::4]
# c_values = ome_raw[0::4]


# x_values = (Linv@ome_raw)[1::4]
# y_values = (Linv@ome_raw)[2::4]
# z_values = (Linv@ome_raw)[3::4]
# c_values = (Linv@ome_raw)[0::4]

# x_values = (ome_raw@Linv)[1::4]
# y_values = (ome_raw@Linv)[2::4]
# z_values = (ome_raw@Linv)[3::4]
# c_values = (ome_raw@Linv)[0::4]

# x_values = V[:,0]
# y_values = V[:,1]
# z_values = V[:,2]
# c_values = 0.*x_values

# fig = plt.figure()
# ax = fig.add_subplot(projection='3d')
# ax.scatter(xs=x_values,ys=y_values,zs=z_values,c=c_values,alpha=0.7,s=10)
# plt.show()

In [ ]:
V_out = ome_out[:,1:]
# V_out = ome_out[:,:3]
# V_out = ome_out[:,:3]

#TODO: plot the resultant point cloud
# x_values = V[:,0]
# y_values = V[:,1]
# z_values = V[:,2]
x_values = V_out[:,0]
y_values = V_out[:,1]
z_values = V_out[:,2]

c_values = 0.*x_values
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(xs=x_values,ys=y_values,zs=z_values,c=c_values,alpha=0.3,s=10)
plt.show()
V_out.shape

In [ ]:
# is the current mesh watertight?
print(f"{mesh_out.is_watertight=}")
# what's the euler number for the mesh?
print(f"{mesh_out.euler_number=}")
# the convex hull is another Trimesh object that is available as a property
# lets compare the volume of our mesh with the volume of its convex hull
print(f"{mesh_out.volume / mesh_out.convex_hull.volume=:.7f}")
# # since the mesh is watertight, it means there is a
# # volumetric center of mass which we can set as the origin for our mesh
print(f"{mesh_out.center_mass=}")
# mesh.vertices -= mesh.center_mass
# print(f"{mesh.center_mass=}")
# what's the moment of inertia for the mesh?
print(f"{mesh_out.moment_inertia=}")

# axis aligned bounding box is available
print(f"{mesh_out.bounding_box.extents=}")

In [ ]:
########################################
# Option #1: do what WJ asked me to do
########################################
# - update uncertainties for simulated annealing fits using 10X epochs instead of 10X more epochs
# - add bigger domain sizes to Figure S3 for the FK model
# - add dotted exponential fit to fig. S2C-D showing tau(N0) scales exponentially with A for N0=2,2s0.


In [ ]:
# # Option #2: aimlessly look for a bug between *** and **** on ub using cupy to do np.linalg.inv
# # TODO: debug mesh_out showing wrong
# # HINT: add more print statements to spin.m and look for the values that do not agree
# # TODO(later): use cupy.linalg.inv
# # - link to doc on ibid: https://docs.cupy.dev/en/stable/reference/generated/cupy.linalg.inv.html
# # HINT: before debugging aimlessly between *** and ****, try np.dot everywhere
# # hint(ub): if doing Option #2, then gpu accelrerate any matrix inversions (only) 
# #TODO(later?): compute matrix a_inv from n-dimensional regular matrix a such that dot(a, a_inv) == eye(n).
# a = cp.array(E)
# a = cp.array(L)
# a_inv = cp.linalg.inv (a)
# n=a_inv.shape[0]
# assert n==a.shape[0]
# assert n==a.shape[1]
# assert (cp.dot(a, a_inv) == eye(n)).get().all()
# Einv = a_inv.get()
# Linv = L_inv.get()

In [ ]:
# invh_banded
A=E
N = np.shape(A)[0]
# D = np.count_nonzero(A[0,:])
D=np.max(np.count_nonzero(A,axis=1))
ab = np.zeros((D,A.shape[0]))
for i in np.arange(1,D):
    ab[i,:] = np.concatenate((np.diag(A,k=i),np.zeros(i,)),axis=None)
ab[0,:] = np.diag(A,k=0)
ab.shape,ab.dtype

# plt.plot(np.count_nonzero(A,axis=1))
for i in np.arange(1,D):
    plt.plot(ab[i],lw=0.5,alpha=0.4,label=f'band #{i}')
# AddLegend()
format_plot(xlabel='Vertex Coordinate',ylabel='Matrix Element')
ax.tick_params(top=True, right=True,direction='in',which='both')
plt.tight_layout()
plt.show()

# plt.plot(np.count_nonzero(A,axis=1))
for i in np.arange(1,D):
    plt.plot(ab[i]**2,lw=0.5,alpha=0.4,label=f'band #{i}')
format_plot(xlabel='Vertex Coordinate',ylabel='Matrix Element Squared')
ax.tick_params(top=True, right=True,direction='in',which='both')
plt.tight_layout()
plt.show()

plt.plot(np.sqrt(np.sum(ab**2,axis=0)), c='k',alpha=0.7,label="(Band-Avg.)")
format_plot(xlabel='Vertex Coordinate',ylabel='RMS Matrix Element')
ax.tick_params(top=True, right=True,direction='in',which='both')
plt.tight_layout()
plt.show()

rmsme = np.sqrt(np.sum(ab**2,axis=0))

# rmsme.shape

alpha=0.5
fontsize=16
fig,ax=plt.subplots(figsize=(9,4))
ax.plot(rmsme[::4], alpha=alpha)#c='k',alpha=0.7)
ax.plot(rmsme[1::4], alpha=alpha)#
ax.plot(rmsme[2::4], alpha=alpha)#
ax.plot(rmsme[3::4], alpha=alpha)#
format_plot(ax=ax,xlabel='Vertex Index',ylabel=r'Band Avg.',fontsize=fontsize)
# ax.set_title("Equal Band Averages in Each Dimension\n"+'\n',fontsize=16)
ax.tick_params(top=True, right=True,direction='in',which='both')
plt.tight_layout()
plt.show()

alpha=0.8
fontsize=16
fig,ax=plt.subplots(figsize=(4,4))
ax.plot(rmsme[::4], alpha=alpha)#c='k',alpha=0.7)
ax.plot(rmsme[1::4]+1, alpha=alpha)#
ax.plot(rmsme[2::4]+2, alpha=alpha)#
ax.plot(rmsme[3::4]+3, alpha=alpha)#
format_plot(ax=ax,xlabel='Vertex Index',ylabel=r'Band Avg. $+ d$',fontsize=fontsize)
ax.set_title("Equal Band Averages in Each Dimension\n"+r"( for $d=0,1,2,3$;  $1:x,2:y,3:z$ )"+'\n',fontsize=16)
ax.tick_params(top=True, right=True,direction='in',which='both')
plt.tight_layout()
plt.show()

In [ ]:
#DONE(failing ^this): try computing L 
#   with right multiplying to get lam
# Q: did ^that work?
# A: not immediately...

In [ ]:
#TODO(failing ^this): make sure E is being constructed correctly?
# Q: are the data points being caste like how L is constructed?
#TODO(am I indexing everyting consistently?)

In [ ]:
#TODO(if ^this works): speed up matrix inversion with scipy.sparse.linalg
#

In [ ]:
#TODO(failing ^this): consider reimplementing using another method
# numpy-quaternion lacks linear algebra solving
# quaternionic may lack linear algebra solving
# complex matrix representation is straightforward and probably faster than the real representation
#note: it's hard to implement quaternions in low-level cuda... but not complex matrix linear algebra...

In [ ]:
#TODO(later): take eigenvalue decomposition of a working example
#TODO: plot the ground state
#TODO: plot the excited states
# boo_real = np.around(np.imag(vals),7)==0
# lam_grnd = np.real(np.min(vals[boo_real]))
# vec_grnd = vecs[:,lam_grnd==vals]
# print(f"{lam_grnd=}")
# # vec_grnd = vecs[:,2]
# assert (np.imag(vec_grnd)==0).all()
# vec_grnd = np.real(vec_grnd)
# plt.plot(vec_grnd)



# scipy.linalg.eig?
# scipy.linalg.eigh?
# scipy.linalg.schur

# # scipy.sparse.linalg.eigs?
# # maxiter =    Maximum number of Arnoldi update iterations allowed
# Notes
# -----
# This function is a wrapper to the ARPACK [1]_ SNEUPD, DNEUPD, CNEUPD,
# ZNEUPD, functions which use the Implicitly Restarted Arnoldi Method to
# find the eigenvalues and eigenvectors [2]_.

# References
# ----------
# .. [1] ARPACK Software, http://www.caam.rice.edu/software/ARPACK/
# .. [2] R. B. Lehoucq, D. C. Sorensen, and C. Yang,  ARPACK USERS GUIDE:
#    Solution of Large Scale Eigenvalue Problems by Implicitly Restarted
#    Arnoldi Methods. SIAM, Philadelphia, PA, 1998.

# # scipy.sparse.linalg.svds
# # scipy.sparse.linalg.eigsh
# vals, vecs = scipy.sparse.linalg.eigs(np.eye(13), k=6)
# print(*vals)
# vals.shape, vecs.shape

# eigvals : eigenvalues of a non-symmetric array.
#^that's not what I want
# eigh : eigenvalues and eigenvectors of a real symmetric or complex
#        Hermitian (conjugate symmetric) array.
#^that's what I want
#eigvalsh : eigenvalues of a real symmetric or complex Hermitian
#            (conjugate symmetric) array.
#^that's not what I want
# scipy.linalg.eig : Similar function in SciPy that also solves the
#                    generalized eigenvalue problem.
#^that's probably slow
# scipy.linalg.schur : Best choice for unitary and other non-Hermitian
#                      normal matrices.
#^that's probably slow

#TODO: find the ground state
# lam_grnd



# TODO(later): determine fastest way to invert a 10000x10000 matrix of quaternions on 1 cpu
- better, TODO: speed up by using complex quaternion representation.

In [ ]:
import quaternion

In [ ]:
# import quaternionic

In [ ]:
# a.shape

# a = np.random.rand(10000, 10000,4)
# qs = np.quaternion(a)
# qs

# qs = a.astype(dtype=np.quaternion)



# a = np.random.rand(10000, 10000*4)
# # a
# # array([[ 0.93138726,  0.46972279,  0.18706385,  0.86605021],
# #        [ 0.70633523,  0.69982741,  0.93303559,  0.61440879],
# #        [ 0.79334456,  0.65912598,  0.0711557 ,  0.46622885],
# #        [ 0.88185987,  0.9391296 ,  0.73670503,  0.27115149],
# #        [ 0.49176628,  0.56688076,  0.13216632,  0.33309146],
# #        [ 0.11951624,  0.86804078,  0.77968826,  0.37229404],
# #        [ 0.33187593,  0.53391165,  0.8577846 ,  0.18336855]])
# qs = quaternion.as_quat_array(a)
# # array([ quaternion(0.931387262880247, 0.469722787598354, 0.187063852060487, 0.866050210100621),
# #        quaternion(0.706335233363319, 0.69982740767353, 0.933035590130247, 0.614408786768725),
# #        quaternion(0.793344561317281, 0.659125976566815, 0.0711557025000925, 0.466228847713644),
# #        quaternion(0.881859869074069, 0.939129602918467, 0.736705031709562, 0.271151494174001),
# #        quaternion(0.491766284854505, 0.566880763189927, 0.132166320200012, 0.333091463422536),
# #        quaternion(0.119516238634238, 0.86804077992676, 0.779688263524229, 0.372294043850009),
# #        quaternion(0.331875925159073, 0.533911652483908, 0.857784598617977, 0.183368547490701)], dtype=quaternion)
# # qsi

# qsi = np.linalg.inv(qs)
# qsi.shape

# # note: quaternion inverted a 10,000x10,000 matrix of quaternions in 3.5 seconds
# # note: quaternion crashes the kernel with 100,000x100,000 matrix
# qsi = 1/qs

# del qsi,qs,a

# a = np.random.rand(10000, 10000,4)
# # a = np.random.normal(size=(17, 11, 4))  # Just some random numbers; last dimension is 4
# q1 = quaternionic.array(a)  # Reinterpret an existing array
# # q2 = quaternionic.array([1.2, 2.3, 3.4, 4.5])  # Create a new array

# # # note: quaternionic inverted a 10,000x10,000 matrix of quaternions in 7 seconds
# # 1/q1
# del q1

# a = np.random.rand(10000*2, 10000*2)

# # # # note: numpy inverted a 20,000x20,000 matrix of real numbers in more than several minutes...
# # np.linalg.inv(a)

# #note: quaternion package crashed the kernel.  does quaternionic crash the kernel?

# np.max(E)


# for c1=1:11
#   cnv=lam;
#   lam=E\lam;
#   lam=lam/norm(lam);
# end

In [ ]:

# res=(E*lam)./lam;  
# fprintf('mean %e, var %e, delta %e\n',mean(res),var(res),norm(cnv-lam))

# L  =sparse(4*nV,4*nV);
# ome=zeros(4*nV,1);
# for c1=1:nT
#   for c2=1:3
#     k0=T(mod(c2-1,3)+1,c1);
#     k1=T(mod(c2+0,3)+1,c1);
#     k2=T(mod(c2+1,3)+1,c1);
#     u1=V(:,k1)-V(:,k0);
#     u2=V(:,k2)-V(:,k0);
#     cta=dot(u1,u2) / norm( cross(u1,u2) );
#     h=jiH([cta*0.5 0 0 0]);
#     ini=[k1*4+plc  k2*4+plc];
#     L(ini,ini)=L(ini,ini)+[ h -h;-h h];
#     if k1>k2
#       k3=k1; k1=k2; k2=k3; % swap
#     end
#     lm1=jiH(lam(k1*4+plc));
#     lm2=jiH(lam(k2*4+plc));
#     edv=jiH([0;V(:,k2)-V(:,k1)]);
#     til=lm1'*edv*lm1/3 + lm1'*edv*lm2/6 + lm2'*edv*lm1/6 + lm2'*edv*lm2/3;
#     ome(k1*4+plc,1)=ome(k1*4+plc,1)-cta*til(:,1)/2;
#     ome(k2*4+plc,1)=ome(k2*4+plc,1)+cta*til(:,1)/2;
#   end
#   if ~mod(c1,500); fprintf('.'); end
# end
# fprintf('\n')

# ome=reshape(ome,[4 nV]);
# ome=ome-repmat(mean(ome,2),[1 nV]);
# ome=reshape(ome,[4*nV 1]);
# ome=L\ome;
# ome=reshape(ome,[4 nV]);
# ome=ome-repmat(mean(ome,2),[1 nV]);
# nrm=sum(ome.*ome,1);
# ome=ome/sqrt(max(nrm));
# V=ome(2:end,:);

# Visualizing solution matrices as heatmaps

In [ ]:
# #4 minutes...
# sns.heatmap(E)

In [ ]:
atol=1e-3
# atol=1e-5
# atol=1e-10
vmax=np.max((-np.min(E),np.max(E)))
boo=np.isclose(E,0.,atol=atol)
frac_nonzero = (~boo).sum()/np.sum(boo)
print(f"percent of matrix elements that are nonzero: {frac_nonzero:.4%}")

In [ ]:
# E[~boo].shape
x,y=boo.nonzero()

np.mean(E[x[::100000],y[::100000]]**2)
# np.mean(E[x[::100000],y[::100000]]*E[x[::100000],y[::100000]])

x[:30]
255857216,
# 100000
x[::100000],y[::100000]

# x,y = k.nonzero()
# fig,ax=plt.s
plt.scatter(x[::100000],y[::100000],s=100,c=E[x[::100000],y[::100000]],vmin=-vmax,vmax=vmax,cmap='bwr') #color as the values in k matrix
# plt.scatter(x,y,s=100,c=E[x,y],vmin=-vmax,vmax=vmax,cmap='bwr') #color as the values in k matrix

In [ ]:
k=E.copy()
k[boo]=np.nan
plt.matshow(k,aspect='auto')

In [ ]:
plt.figure(figsize=(20,15))
ax=subplot(111)
# sns.heatmap(corr,ax=ax)

In [ ]:
# sns.heatmap(L)?

In [ ]:
L.shape

In [ ]:
k[k==0.0]=np.nan
plt.matshow(k,aspect='auto')

In [ ]:
x,y = k.nonzero()
plt.scatter(x,y,s=100,c=k[x,y]) #color as the values in k matrix

In [ ]:
sns.heatmap(
    E,,L
    vmin=None,
    vmax=None,
    cmap=None,
    center=None,
    robust=False,
    annot=None,
    fmt='.2g',
    annot_kws=None,
    linewidths=0,
    linecolor='white',
    cbar=True,
    cbar_kws=None,
    cbar_ax=None,
    square=False,
    xticklabels='auto',
    yticklabels='auto',
    mask=None,
    ax=None,
    **kwargs,
)